In [1]:
import functools
import re

import numpy as np
import pandas as pd
from string_grouper import match_most_similar

import auxiliar_4

# Preparación de datos

## Datos principales

In [2]:
datos = pd.read_csv('datos/zoom1/zoom1_anon_completo.csv')
datos

,id,submitdate,lastpage,startlanguage,seed,startdate,datestamp,encuestador,encuestador[other],conset,...,y65[SQ002],y65[SQ003],y65[SQ004],y65[SQ005],y65[SQ006],y65[SQ007],y65[SQ008],y65[SQ009],y65[SQ010],sdfds3
0,67,2021-03-27 13:32:28,5,es-MX,909191565,2021-03-27 13:03:10,2021-03-27 13:32:28,A13,NaN,Y,...,NECESARIO,IMPORTANTE,SOCIAL,PROTECCION,AUTOCUIDADO,FAMILIA,SALUD,COVID,HOSPITAL,10.0
1,68,2021-03-27 19:38:43,5,es-MX,99489830,2021-03-27 19:07:21,2021-03-27 19:38:43,A13,NaN,Y,...,ESTIMULACION,NEURONAL,CONEXION,LONGEVIDAD,FORTALEZA,NaN,NaN,NaN,NaN,6.0
2,69,2021-03-29 11:47:46,5,es-MX,1592450376,2021-03-29 11:04:42,2021-03-29 11:47:46,A07,NaN,Y,...,HOSPITAL,ENFERMEDAD,COVID,PANDEMIA,NECESARIO,UTIL,INCOMODO,NaN,NaN,8.0
3,70,2021-03-29 12:37:20,5,es-MX,762883078,2021-03-29 12:04:01,2021-03-29 12:37:20,A04,NaN,Y,...,virus,pandemia,situacion,global,estres,ansiedad,muertes,NaN,NaN,8.0
4,71,2021-03-29 14:33:34,5,es-MX,1746037044,2021-03-29 14:00:48,2021-03-29 14:33:34,A03,NaN,Y,...,PANDEMIA,PREVENCION,ENFERMEDAD,CUARENTENA,NECESARIO,SALUD,INCOMODO,NaN,NaN,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
718,851,2022-10-29 17:28:54,5,es-MX,584177600,2022-10-29 17:07:05,2022-10-29 17:28:54,A05,NaN,Y,...,MALA ALIMENTACION,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0
719,852,2022-10-29 18:20:26,5,es-MX,921555126,2022-10-29 18:02:41,2022-10-29 18:20:26,A05,NaN,Y,...,CIENCIA,TERMINO,AUTOCUIDADO,NaN,NaN,NaN,NaN,NaN,NaN,4.0
720,853,2022-10-29 20:23:28,5,es-MX,114360708,2022-10-29 20:02:20,2022-10-29 20:23:28,A05,NaN,Y,...,DISCIPLINA,SALUD,BUENO,CONSCIENCIA,RIGUROSO,TIEMPO,NaN,NaN,NaN,7.0
721,854,2022-10-31 19:02:54,5,es-MX,2117179292,2022-10-31 18:32:54,2022-10-31 19:02:54,A11,NaN,Y,...,bienestar,constancia,cuerpo,sudor,autoestima,reto,atención,diversión,paz,10.0


## Diccionario de datos

El diccionario contiene la información sobre las preguntas de la encuesta: su código, sus posibles respuestas y descripción –entre otros detalles.
Como los datos se encuetran en `.csv` se genera irregularidad en las columnas del marco de datos resultante:

In [3]:
diccionario = pd.read_csv('datos/zoom1/zoom1-dicc-tx.csv')
diccionario

,item order,related,alive,new_code,code,question,options,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 62,Unnamed: 63,Unnamed: 64,Unnamed: 65,Unnamed: 66,Unnamed: 67,Unnamed: 68,Unnamed: 69,Unnamed: 70,Unnamed: 71
0,1.0,G,_,encuestador,encuestador,Encuestador,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2.0,G,_,encuestador_otro,encuestador[other],Encuestador_otro,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3.0,G,_,consentimiento_leido,conset,¿Leíste el concentimiento informado?,Y = Sí_1,N = No_2,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4.0,G,_,consentimiento_aceptado,q35,¿Aceptas el concentimiento informado?,Y = Sí_1,N = No_2,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5.0,G,_,camara_encendida,q2941,¿Aceptas ser entrevistado con la cámara encend...,Y = Sí_1,N = No_2,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
994,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
995,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
997,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


El diccionario tiene una columna con el código de cada pregunta, el cual corresponde al nombre de cada columna del conjunto de datos principal.
Para simplificar el análisis posterior, se utiliza este código como índice del diccionario:

In [4]:
diccionario.set_index('code', inplace=True)
diccionario

,item order,related,alive,new_code,question,options,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,...,Unnamed: 62,Unnamed: 63,Unnamed: 64,Unnamed: 65,Unnamed: 66,Unnamed: 67,Unnamed: 68,Unnamed: 69,Unnamed: 70,Unnamed: 71
code,,,,,,,,,,,,,,,,,,,,,
encuestador,1.0,G,_,encuestador,Encuestador,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
encuestador[other],2.0,G,_,encuestador_otro,Encuestador_otro,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
conset,3.0,G,_,consentimiento_leido,¿Leíste el concentimiento informado?,Y = Sí_1,N = No_2,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
q35,4.0,G,_,consentimiento_aceptado,¿Aceptas el concentimiento informado?,Y = Sí_1,N = No_2,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
q2941,5.0,G,_,camara_encendida,¿Aceptas ser entrevistado con la cámara encend...,Y = Sí_1,N = No_2,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Mapeo de preguntas y respuestas

A cada pregunta del diccionario, este mapeo le asigna el conjunto de sus posibles respuestas de opción múltiple (en caso de que las tenga).

In [5]:
mapeo_respuestas = auxiliar_4.construir_mapeo_preguntas_y_respuestas(diccionario)
mapeo_respuestas

{'conset': {'Y': 'Sí_1', 'N': 'No_2'},
 'q35': {'Y': 'Sí_1', 'N': 'No_2'},
 'q2941': {'Y': 'Sí_1', 'N': 'No_2'},
 'q8588': {'Y': 'Sí_1', 'N': 'No_2'},
 'Sexo_zoom1': {'F': 'Mujer_1', 'M': 'Hombre_2'},
 'Campus': {'A1': 'UNAM CIUDAD UNIVERSITARIA_1',
  'A2': 'UNAM CAMPUS ZARAGOZA_2',
  'A3': 'IBEROAMERICANA SANTA FE_3',
  'A4': 'UNIVERSIDAD PANAMERICANA_4',
  'A5': 'UNIVERSIDAD DE VALLE DE MÉXICO_5',
  'A6': 'UNIVERSIDAD ANÁHUAC_6',
  'A7': 'UNIVERSIDAD DE GUANAJUATO_7',
  'A8': 'UANL CIENCIAS DE LA SALUD_8'},
 'Carrera': {'A1': 'Biología_1',
  'A2': 'Ciencias De La Computación_2',
  'A3': 'Ciencias De La Comunicación (Periodismo)_3',
  'A4': 'Ciencias De La Tierra_4',
  'A5': 'Física_5',
  'A6': 'Ingeniería Biomédica_6',
  'A7': 'Médico Cirujano_7',
  'A8': 'Psicología_8'},
 'CarreraCU': {'A2': 'Actuaría_1',
  'A3': 'Arqitectura_2',
  'A4': 'Arqitectura del Paisaje_3',
  'A5': 'Ciencias de la Computación_4',
  'A6': 'Diseño Industrial_5',
  'A7': 'Física_6',
  'A8': 'Ingeniería Civil_7

## Funciones auxiliares

El propósito de las funciones auxiliares es simplificar la cantidad de código que se escribe.

La definición de estas funciones se encuentra en el módulo `auxiliar.py`.

La mayoría de estas funciones reciben como parámetros el conjunto de datos y el mapeo de preguntas y respuestas. En este cuaderno solo se tiene un conjunto de datos (zoom1) y un mapeo de preguntas y respuestas. Por lo tanto, cada vez que se llame a estas funciones durante el análisis se tendría que escribir el nombre del conjunto de datos y el mapeo.

Para evitar tanta repetición y permitir que las funciones auxiliares se puedan usar en otros cuadernos, *se creará nuevas funciones que simplemente llamen a las funciones auxiliares, pero con los parámetros de datos y de mapeo* **fijados**: esto se logra mediante el uso de `functools.partial`, que recibe una función y los parámetros que se le quieren fijar y devuelve una nueva función con los parámetros fijados.

De este modo, las nuevas funciones tienen el mismo comportamiento pero reciben menos parámetros.

In [6]:
mapear_codigos = functools.partial(auxiliar_4.mapear_codigos, datos, mapeo_respuestas)
llenar_respuesta_libre = functools.partial(auxiliar_4.llenar_respuesta_libre, datos)
llenar_vacios = functools.partial(auxiliar_4.llenar_vacios, datos)
mostrar_unicos = functools.partial(auxiliar_4.mostrar_unicos, datos)
contar_faltantes = functools.partial(auxiliar_4.contar_faltantes, datos)
unificar_observaciones = functools.partial(auxiliar_4.unificar_observaciones, datos)

# Limpieza general

En esta sección se limpiará la mayoría de  las columnas del conjunto de datos principal.

Como cada una requiere un tratamiento diferente, se dedicará una subsección individual para cada una.

El orden en el que se limpiarán las columnas es en el que aparecen en el conjunto de datos, no en el que se presentan en el diccionario.

## Descripción proceso

### Columnas con sufijo
Muchas columnas son preguntas de opción múltiple, pero en ocasiones se permite que el encuestador de una respuesta libre.
En este caso, se tiene una nueva columna con el mismo nombre de la columna original pero con un sufijo (por lo común `[other]`).
 Por ejemplo, si la columna `campus` permite que el encuestado escriba una respuesta libre, entonces se tendrá una columna `campus[other]` con la respuesta libre del encuestado.

En caso de que el encuestado dé una respuesta libre, la columna original tendrá un valor faltante (o tendrá algún valor como `-oth-`) y la columna con el sufijo tendrá el valor de la respuesta libre.

Para limpiar estas columnas, se debe:
- Unificar ambas columnas.
    - Llenar los valores faltantes de la columna original con el valor de la columna con el sufijo.
    - Llenar los valores `-oth-` de la columna original con el valor de la columna con el sufijo.
- Eliminar las columnas con el sufijo.

Los pasos anteriores se realizan con la función `llenar_respuesta_libre`.

### Valores faltantes
Se usan diferentes códigos para ciertos tipos de valores faltantes:
- B1: no aplica porque algunas personas ingresaron a la carrera antes de la pandemia y otras durante la pandemia.
- B2: valor perdido
- B3: problema en lime survey

## Columnas elementales

### Encuestados sin información

El encuestado con el `ID` 288 tiene casi todas sus columnas vacías, así que se eliminará su registro.

In [7]:
datos.drop(index=datos[datos['id'] == 288].index, inplace=True)

### Encuestador

In [8]:
mostrar_unicos('encuestador')

0     A02
1     A03
2     A04
3     A05
4     A06
5     A07
6     A08
7     A09
8     A10
9     A11
10    A12
11    A13
12    A14
13    A15
14    A16
15    A17
16    A18
17    A19
18    A20
19    A21
20    A22
21    A23
22    NaN
Name: encuestador, dtype: object

In [9]:
contar_faltantes('encuestador')

7

In [10]:
llenar_respuesta_libre('encuestador')
llenar_vacios('encuestador', '00')
contar_faltantes('encuestador')

/Users/erickavendanogarcia/Downloads/servicio_social/encuestas/auxiliar_4.py:143: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[columna].fillna(valor, inplace=True)


0

### conset

In [11]:
contar_faltantes('conset')

1

In [12]:
# Observamos que la variable conset tiene dos valores faltantes, en este caso los llenamos con 'no'.
# Es mejor prevenir que lamentar.

llenar_vacios('conset', 'N')
contar_faltantes('conset')

0

### q35

In [13]:
# La columna q35 es el consentimiento aceptado, en este caso también hay valores faltantes,
# los cuales vamos a establecer como un 'no'
llenar_vacios('q35', 'N')

### q2941 y q8588

In [14]:
# La columna q2941 es sobre si la camara estaba encendida, y q8588 sobre si la reunión se podía grabar;
# los valores faltantes se ponen como 'no'.

llenar_vacios('q2941', 'N')
llenar_vacios('q8588', 'N')

### folio

In [15]:
# Existen valores faltantes en la columna folio. Los voy a llenar con un folio predeterminado '0'.
# También hay valores faltantes en el sexo, en este caso van a llenarse con un 'Desconocido'."
llenar_vacios('folio', 0)
contar_faltantes('folio')

0

### Sexo

In [16]:
llenar_vacios('Sexo', 'Desconocido')

In [17]:
contar_faltantes('Sexo')

0

### Dirección

In [18]:
# Existen valores faltantes en las dirreciones Direccion[SQ008], ¿cómo podemos solucionarlo? Le pondré un 0.
llenar_vacios('Direccion[SQ008]', 0)

## Campus

In [19]:
mostrar_unicos('Campus')

0    -oth-
1       A1
2       A2
3       A3
4       A5
5       A7
6       A8
Name: Campus, dtype: object

Primero se extrae los valores de las claves:

In [20]:
mapear_codigos('Campus')
mostrar_unicos('Campus')

/Users/erickavendanogarcia/Downloads/servicio_social/encuestas/auxiliar_4.py:99: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[columna].replace(mapeo[columna], inplace=True)


0                               -oth-
1           IBEROAMERICANA SANTA FE_3
2         UANL CIENCIAS DE LA SALUD_8
3              UNAM CAMPUS ZARAGOZA_2
4         UNAM CIUDAD UNIVERSITARIA_1
5         UNIVERSIDAD DE GUANAJUATO_7
6    UNIVERSIDAD DE VALLE DE MÉXICO_5
Name: Campus, dtype: object

In [21]:
llenar_respuesta_libre('Campus')
mostrar_unicos('Campus')

0                  Ciencias de la salud 
1                                    DCS
2                           FES ZARAGOZA
3                            GUANAJUATO 
4                                  IBERO
5              IBEROAMERICANA SANTA FE_3
6          U. DE GUANAJUATO CAMPUS LEON 
7           U. de Guanajuato Campus LEON
8          U. de Guanajuato Campus Leon 
9          U. de Guanajuato Campus León 
10           UANL CIENCIAS DE LA SALUD_8
11                                  UGTO
12                UNAM CAMPUS ZARAGOZA_2
13           UNAM CIUDAD UNIVERSITARIA_1
14                                 UNISA
15                                UNISA 
16              UNIVERSIDA DE GUANAJUATO
17             UNIVERSIDAD DE GUANAJUATO
18           UNIVERSIDAD DE GUANAJUATO_7
19      UNIVERSIDAD DE VALLE DE MÉXICO_5
20    Universidad Guanajuato Campus LEON
21                   Villa de los niños 
22                                 unisa
Name: Campus, dtype: object

In [22]:
datos['Campus'] = datos['Campus'].str.strip().str.lower()
mostrar_unicos('Campus')

0                   ciencias de la salud
1                                    dcs
2                           fes zaragoza
3                             guanajuato
4                                  ibero
5              iberoamericana santa fe_3
6           u. de guanajuato campus leon
7           u. de guanajuato campus león
8            uanl ciencias de la salud_8
9                                   ugto
10                unam campus zaragoza_2
11           unam ciudad universitaria_1
12                                 unisa
13              universida de guanajuato
14             universidad de guanajuato
15           universidad de guanajuato_7
16      universidad de valle de méxico_5
17    universidad guanajuato campus leon
18                    villa de los niños
Name: Campus, dtype: object

Se nota que hay algunas respuestas que corresponden a la misma institución, pero escritas de manera diferente. Se unificará los distintos formatos para la universidad de Guanajuato.

In [23]:
datos['Campus'][datos['Campus'].str.contains('guanajuato', case=False)]

548                            guanajuato
549    universidad guanajuato campus leon
550          u. de guanajuato campus leon
551             universidad de guanajuato
552              universida de guanajuato
553          u. de guanajuato campus león
554             universidad de guanajuato
555             universidad de guanajuato
558             universidad de guanajuato
559          u. de guanajuato campus leon
560          u. de guanajuato campus leon
562             universidad de guanajuato
564             universidad de guanajuato
583             universidad de guanajuato
714           universidad de guanajuato_7
715           universidad de guanajuato_7
716           universidad de guanajuato_7
717           universidad de guanajuato_7
718           universidad de guanajuato_7
719           universidad de guanajuato_7
720           universidad de guanajuato_7
721           universidad de guanajuato_7
722           universidad de guanajuato_7
Name: Campus, dtype: object

In [24]:
unificar_observaciones('Campus', r'.*guanajuato.*|ugto', 'universidad de guanajuato')
unificar_observaciones('Campus', r'ibero', 'iberoamericana santa fe_3')
unificar_observaciones('Campus', r'zaragoza', 'unam campus zaragoza_2')
unificar_observaciones('Campus', r'^dcs|salud$', 'ciencias de la salud')
datos['Campus'].value_counts()

Campus
unam ciudad universitaria_1         316
iberoamericana santa fe_3           207
uanl ciencias de la salud_8          93
unam campus zaragoza_2               67
universidad de guanajuato            24
unisa                                11
ciencias de la salud                  2
universidad de valle de méxico_5      1
villa de los niños                    1
Name: count, dtype: int64

El campus ciencias de la salud pertenece a la universidad autónoma de Nuevo León (UANL).
Se requiere un código para universidad de Guanajuato y otro para ciencias de la salud en UANL

In [25]:
unificar_observaciones('Campus', r'universidad de guanajuato', 'universidad de guanajuato_7')
unificar_observaciones('Campus', r'ciencias de la salud', 'uanl ciencias de la salud_8')
unificar_observaciones('Campus', r'unisa', 'Universidad de la Salud de la Ciudad de México_9')
unificar_observaciones('Campus', r'villa de los niños', 'villa de los niños_10')
unificar_observaciones('Campus', r'^a[78]', 'desconocido_0')
mostrar_unicos('Campus')

0    Universidad de la Salud de la Ciudad de México_9
1                           iberoamericana santa fe_3
2                         uanl ciencias de la salud_8
3                              unam campus zaragoza_2
4                         unam ciudad universitaria_1
5                         universidad de guanajuato_7
6                    universidad de valle de méxico_5
7                               villa de los niños_10
Name: Campus, dtype: object

### Códigos de campus

In [26]:
prefijo = 'A'
codigos_campus = {}
for campus in mostrar_unicos('Campus'):
    posicion = re.findall(r'\d+', campus)[0]
    codigo = prefijo + posicion
    codigos_campus[codigo] = campus.upper()
codigos_campus

{'A9': 'UNIVERSIDAD DE LA SALUD DE LA CIUDAD DE MÉXICO_9',
 'A3': 'IBEROAMERICANA SANTA FE_3',
 'A8': 'UANL CIENCIAS DE LA SALUD_8',
 'A2': 'UNAM CAMPUS ZARAGOZA_2',
 'A1': 'UNAM CIUDAD UNIVERSITARIA_1',
 'A7': 'UNIVERSIDAD DE GUANAJUATO_7',
 'A5': 'UNIVERSIDAD DE VALLE DE MÉXICO_5',
 'A10': 'VILLA DE LOS NIÑOS_10'}

In [27]:
codigos_campus.update(mapeo_respuestas['Campus'])
codigos_campus

{'A9': 'UNIVERSIDAD DE LA SALUD DE LA CIUDAD DE MÉXICO_9',
 'A3': 'IBEROAMERICANA SANTA FE_3',
 'A8': 'UANL CIENCIAS DE LA SALUD_8',
 'A2': 'UNAM CAMPUS ZARAGOZA_2',
 'A1': 'UNAM CIUDAD UNIVERSITARIA_1',
 'A7': 'UNIVERSIDAD DE GUANAJUATO_7',
 'A5': 'UNIVERSIDAD DE VALLE DE MÉXICO_5',
 'A10': 'VILLA DE LOS NIÑOS_10',
 'A4': 'UNIVERSIDAD PANAMERICANA_4',
 'A6': 'UNIVERSIDAD ANÁHUAC_6'}

In [28]:
mapeo_inverso_campus = {v.lower(): k for k, v in codigos_campus.items()}
mapeo_inverso_campus

{'universidad de la salud de la ciudad de méxico_9': 'A9',
 'iberoamericana santa fe_3': 'A3',
 'uanl ciencias de la salud_8': 'A8',
 'unam campus zaragoza_2': 'A2',
 'unam ciudad universitaria_1': 'A1',
 'universidad de guanajuato_7': 'A7',
 'universidad de valle de méxico_5': 'A5',
 'villa de los niños_10': 'A10',
 'universidad panamericana_4': 'A4',
 'universidad anáhuac_6': 'A6'}

In [29]:
datos['Campus'].replace(mapeo_inverso_campus, inplace=True)
mostrar_unicos('Campus')

/var/folders/hb/c3zh8bns0mx4nw98_6k7rx8m0000gn/T/ipykernel_4464/1835897616.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  datos['Campus'].replace(mapeo_inverso_campus, inplace=True)


0                                                  A1
1                                                 A10
2                                                  A2
3                                                  A3
4                                                  A5
5                                                  A7
6                                                  A8
7    Universidad de la Salud de la Ciudad de México_9
Name: Campus, dtype: object

La columna `otrocampus` se utilizará en la siguiente subsección para lidiar con las carreras.

In [30]:
lista_campus = []
for codigo, campus in codigos_campus.items():
    lista_campus.append(codigo + ' = ' + campus)
lista_campus.sort()
lista_campus

['A1 = UNAM CIUDAD UNIVERSITARIA_1',
 'A10 = VILLA DE LOS NIÑOS_10',
 'A2 = UNAM CAMPUS ZARAGOZA_2',
 'A3 = IBEROAMERICANA SANTA FE_3',
 'A4 = UNIVERSIDAD PANAMERICANA_4',
 'A5 = UNIVERSIDAD DE VALLE DE MÉXICO_5',
 'A6 = UNIVERSIDAD ANÁHUAC_6',
 'A7 = UNIVERSIDAD DE GUANAJUATO_7',
 'A8 = UANL CIENCIAS DE LA SALUD_8',
 'A9 = UNIVERSIDAD DE LA SALUD DE LA CIUDAD DE MÉXICO_9']

In [31]:
cadena_campus = ','.join(lista_campus)
cadena_campus

'A1 = UNAM CIUDAD UNIVERSITARIA_1,A10 = VILLA DE LOS NIÑOS_10,A2 = UNAM CAMPUS ZARAGOZA_2,A3 = IBEROAMERICANA SANTA FE_3,A4 = UNIVERSIDAD PANAMERICANA_4,A5 = UNIVERSIDAD DE VALLE DE MÉXICO_5,A6 = UNIVERSIDAD ANÁHUAC_6,A7 = UNIVERSIDAD DE GUANAJUATO_7,A8 = UANL CIENCIAS DE LA SALUD_8,A9 = UNIVERSIDAD DE LA SALUD DE LA CIUDAD DE MÉXICO_9'

## Carreras

Antes de combinar las columnas correspondientes a las carreras, vamos a mapearlas conforme a su
respectivo diccionario, como lo hicimos en el caso de las opciones de la columna Campus, esto debido a que la
carrera A1 en un campus podría no ser igual a la carrera A1 en otro campus: ejemplo, la carrera A1 en FES Zaragoza
podría ser "Psicología", mientras que en CU podría ser "Ingeniería"

In [32]:
columnas_carreras = datos.columns[datos.columns.str.contains('Carrera\w+$')]
columnas_carreras

<>:1: SyntaxWarning: invalid escape sequence '\w'
<>:1: SyntaxWarning: invalid escape sequence '\w'
/var/folders/hb/c3zh8bns0mx4nw98_6k7rx8m0000gn/T/ipykernel_4464/1785319939.py:1: SyntaxWarning: invalid escape sequence '\w'
  columnas_carreras = datos.columns[datos.columns.str.contains('Carrera\w+$')]


Index(['CarreraCU', 'CarreraZaragoza', 'CarreraIbero', 'Carreraotrasunis'], dtype='object')

In [33]:
datos[columnas_carreras]

,CarreraCU,CarreraZaragoza,CarreraIbero,Carreraotrasunis
0,NaN,NaN,A29,NaN
1,NaN,NaN,A29,NaN
2,NaN,NaN,A29,NaN
3,NaN,NaN,A29,NaN
4,NaN,NaN,A29,NaN
...,...,...,...,...
718,NaN,NaN,NaN,NaN
719,NaN,NaN,NaN,NaN
720,NaN,NaN,NaN,NaN
721,NaN,NaN,NaN,NaN


In [34]:
mostrar_unicos(columnas_carreras[0])

0    -oth-
1      A23
2      A26
3      A27
4      A32
5       A5
6       A7
7      NaN
Name: CarreraCU, dtype: object

In [35]:
mostrar_unicos(columnas_carreras[1])

0     A9
1    NaN
Name: CarreraZaragoza, dtype: object

In [36]:
mostrar_unicos(columnas_carreras[2])

0    A15
1    A29
2    NaN
Name: CarreraIbero, dtype: object

In [37]:
datos['Carreraotrasunis'].value_counts()

Carreraotrasunis
A3    63
A2    42
A1    20
Name: count, dtype: int64

In [38]:
mostrar_unicos(columnas_carreras[3])

0     A1
1     A2
2     A3
3    NaN
Name: Carreraotrasunis, dtype: object

In [39]:
for col in columnas_carreras:
    mapear_codigos(col)
    try:
        llenar_respuesta_libre(col)
    except:
        pass

/Users/erickavendanogarcia/Downloads/servicio_social/encuestas/auxiliar_4.py:99: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[columna].replace(mapeo[columna], inplace=True)
/Users/erickavendanogarcia/Downloads/servicio_social/encuestas/auxiliar_4.py:99: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values alway

In [40]:
mostrar_unicos(columnas_carreras[0])

0                                    Biología_22
1                   Ciencias de la Computación_4
2    Ciencias de la Comunicación (Periodismo)_31
3                                       Física_6
4                             Médico Cirujano_25
5                                  Psicología_26
6                          ciencias de la tierra
7                         ciencias de la tierra 
8                                            NaN
Name: CarreraCU, dtype: object

In [41]:
mostrar_unicos(columnas_carreras[1])

0    Psicología_8
1             NaN
Name: CarreraZaragoza, dtype: object

In [42]:
mostrar_unicos(columnas_carreras[2])

0    Ingeniería Biomédica_14
1              Psicología_28
2                        NaN
Name: CarreraIbero, dtype: object

In [43]:
mostrar_unicos(columnas_carreras[3])

0              A2
1              A3
2    Psicología_1
3             NaN
Name: Carreraotrasunis, dtype: object

### Combinar columnas

Aquí se combinan todas las columnas que sobre carreras. Se pierde información sobre el código particular de cada campus, pero tras recodificar se puede recuperar el campus a través de la columna `Campus`.

Se inicializa la serie de carreras:

In [44]:
carreras = pd.Series(dtype='str', index=datos.index)
carreras

0      NaN
1      NaN
2      NaN
3      NaN
4      NaN
      ... 
718    NaN
719    NaN
720    NaN
721    NaN
722    NaN
Length: 722, dtype: object

In [45]:
for col_carrera in datos.filter(regex='Carrera\w+$').columns:
    carreras = carreras.combine_first(datos[col_carrera])
datos.drop(columns=columnas_carreras, inplace=True)
carreras.drop_duplicates().sort_values()

<>:1: SyntaxWarning: invalid escape sequence '\w'
<>:1: SyntaxWarning: invalid escape sequence '\w'
/var/folders/hb/c3zh8bns0mx4nw98_6k7rx8m0000gn/T/ipykernel_4464/2132380260.py:1: SyntaxWarning: invalid escape sequence '\w'
  for col_carrera in datos.filter(regex='Carrera\w+$').columns:


610                                             A2
609                                             A3
561                                    Biología_22
566                   Ciencias de la Computación_4
569    Ciencias de la Comunicación (Periodismo)_31
571                                       Física_6
150                        Ingeniería Biomédica_14
5                               Médico Cirujano_25
8                                     Psicología_1
278                                  Psicología_26
0                                    Psicología_28
455                                   Psicología_8
573                          ciencias de la tierra
568                         ciencias de la tierra 
714                                            NaN
dtype: object

Todos los datos de la columna `otrocampus` son relacionados a la carrera que se estudia, por este motivo se añadirán a la serie de carreras:

In [46]:
carreras = carreras.combine_first(datos['otrocampus'])
datos.drop(columns='otrocampus', inplace=True)
carreras.drop_duplicates().sort_values()

610                                             A2
609                                             A3
561                                    Biología_22
566                   Ciencias de la Computación_4
569    Ciencias de la Comunicación (Periodismo)_31
571                                       Física_6
150                        Ingeniería Biomédica_14
5                               Médico Cirujano_25
8                                     Psicología_1
278                                  Psicología_26
0                                    Psicología_28
455                                   Psicología_8
573                          ciencias de la tierra
568                         ciencias de la tierra 
714                                            NaN
dtype: object

In [47]:
carreras.isna().sum()

9

Se tiene que añadir la columna de carrera justo al lado de la de campus.

Primero se determina el índice de la columna campus y se verifica que sea el correcto:

In [48]:
indice_campus = datos.columns.get_loc('Campus')
indice_campus

16

In [49]:
datos.columns[indice_campus]

'Campus'

Ahora se inserta la serie

In [50]:
datos.insert(indice_campus + 1, 'Carrera', carreras)

In [51]:
unificar_observaciones('Carrera', r'psicología', 'Psicología')
unificar_observaciones('Carrera', r'tierra', 'Ciencias de la tierra')
unificar_observaciones('Carrera', r'Médico Cirujano', 'Medicina')
datos['Carrera'].value_counts()

Carrera
Psicología                                     470
Medicina                                       124
A3                                              63
A2                                              42
Biología_22                                      6
Ingeniería Biomédica_14                          2
Ciencias de la tierra                            2
Física_6                                         2
Ciencias de la Computación_4                     1
Ciencias de la Comunicación (Periodismo)_31      1
Name: count, dtype: int64

In [52]:
datos['Carrera'] = datos['Carrera'].str.replace(r'_\d+', '', regex=True).str.lower()
mostrar_unicos('Carrera')

0                                           a2
1                                           a3
2                                     biología
3                   ciencias de la computación
4     ciencias de la comunicación (periodismo)
5                        ciencias de la tierra
6                                       física
7                         ingeniería biomédica
8                                     medicina
9                                   psicología
10                                         NaN
Name: Carrera, dtype: object

### Códigos de carrera

In [53]:
sufijo = 'A'
mapeo_carrera = {}
for i in range(len(mostrar_unicos('Carrera'))):
    posicion = str(i + 1)
    codigo = sufijo + posicion
    carrera = mostrar_unicos('Carrera')[i]
    mapeo_carrera[codigo] = carrera
mapeo_carrera['A10'] = 'desconocido'
mapeo_carrera

{'A1': 'a2',
 'A2': 'a3',
 'A3': 'biología',
 'A4': 'ciencias de la computación',
 'A5': 'ciencias de la comunicación (periodismo)',
 'A6': 'ciencias de la tierra',
 'A7': 'física',
 'A8': 'ingeniería biomédica',
 'A9': 'medicina',
 'A10': 'desconocido',
 'A11': nan}

In [54]:
mapeo_inverso_carrera = {v: k for k, v in mapeo_carrera.items()}
mapeo_inverso_carrera

{'a2': 'A1',
 'a3': 'A2',
 'biología': 'A3',
 'ciencias de la computación': 'A4',
 'ciencias de la comunicación (periodismo)': 'A5',
 'ciencias de la tierra': 'A6',
 'física': 'A7',
 'ingeniería biomédica': 'A8',
 'medicina': 'A9',
 'desconocido': 'A10',
 nan: 'A11'}

In [55]:
llenar_vacios('Carrera', 'desconocido')
datos['Carrera'].replace(mapeo_inverso_carrera, inplace=True)
mostrar_unicos('Carrera')

/Users/erickavendanogarcia/Downloads/servicio_social/encuestas/auxiliar_4.py:143: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[columna].fillna(valor, inplace=True)


0             A1
1            A10
2             A2
3             A3
4             A4
5             A5
6             A6
7             A7
8             A8
9             A9
10    psicología
Name: Carrera, dtype: object

In [56]:
lista_mapeo_carreras = []
for codigo, carrera in mapeo_carrera.items():
    lista_mapeo_carreras.append(codigo + ' = ' + str(carrera).title())
lista_mapeo_carreras.sort()
lista_mapeo_carreras

['A1 = A2',
 'A10 = Desconocido',
 'A11 = Nan',
 'A2 = A3',
 'A3 = Biología',
 'A4 = Ciencias De La Computación',
 'A5 = Ciencias De La Comunicación (Periodismo)',
 'A6 = Ciencias De La Tierra',
 'A7 = Física',
 'A8 = Ingeniería Biomédica',
 'A9 = Medicina']

In [57]:
cadena_carrera = ','.join(lista_mapeo_carreras)
cadena_carrera

'A1 = A2,A10 = Desconocido,A11 = Nan,A2 = A3,A3 = Biología,A4 = Ciencias De La Computación,A5 = Ciencias De La Comunicación (Periodismo),A6 = Ciencias De La Tierra,A7 = Física,A8 = Ingeniería Biomédica,A9 = Medicina'

## Semestre

In [58]:
# La variable Semestre tiene valores faltantes, los cuales llenaremos con 'A0'
llenar_vacios('Semestre', 'A0')


## dbs34

In [59]:
# La columna dbs34 es el número de cuenta de su universidad, pondremos 0 en los valores faltantes.
llenar_vacios('dbs34', '0')


/Users/erickavendanogarcia/Downloads/servicio_social/encuestas/auxiliar_4.py:143: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df[columna].fillna(valor, inplace=True)


## FechaDeNacimiento

In [60]:
datos['FechaDeNacimiento'] = pd.to_datetime(datos['FechaDeNacimiento'], infer_datetime_format=True)

/var/folders/hb/c3zh8bns0mx4nw98_6k7rx8m0000gn/T/ipykernel_4464/4120569437.py:1: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  datos['FechaDeNacimiento'] = pd.to_datetime(datos['FechaDeNacimiento'], infer_datetime_format=True)


## EdoCivil

In [61]:
llenar_respuesta_libre('EdoCivil')
# TODO: es válido crear un nuevo código A0?
# Sustituimos los nan restantes por A0
llenar_vacios('EdoCivil', 'A0')
mostrar_unicos('EdoCivil')

/Users/erickavendanogarcia/Downloads/servicio_social/encuestas/auxiliar_4.py:143: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[columna].fillna(valor, inplace=True)


0          A2
1          A3
2          A4
3          A6
4    SEPARADA
5    Separada
Name: EdoCivil, dtype: object

In [62]:
# TODO: es válido crear un nuevo código A7?
unificar_observaciones('EdoCivil', r'separad[oa]', 'A7')
mostrar_unicos('EdoCivil')

0    A2
1    A3
2    A4
3    A6
4    A7
Name: EdoCivil, dtype: object

## Promedio

La columna `promedio` tiene el promedio del semestre anterior.

In [63]:
mostrar_unicos('Promedio').sort_values(ascending=False)

53    958.0
52    886.0
51     98.0
50     97.0
49     96.0
48     95.0
47     94.0
46     93.0
45     92.0
44     91.0
43     90.0
42     89.0
41     88.0
40     86.0
39     85.0
38     84.0
37     83.0
36     82.0
35     10.0
34      9.9
33      9.8
32      9.7
31      9.6
30      9.5
29      9.4
28      9.3
27      9.2
26      9.1
25      9.0
24      8.9
23      8.8
22      8.7
21      8.6
20      8.5
19      8.4
18      8.3
17      8.2
16      8.1
15      8.0
14      7.9
13      7.8
12      7.7
11      7.6
10      7.5
9       7.4
8       7.3
7       7.2
6       7.1
5       7.0
4       6.8
3       6.3
2       6.0
1       4.0
0       0.0
54      NaN
Name: Promedio, dtype: float64

In [64]:
contar_faltantes('Promedio')

9

Como hay valores mayores a 10 se necesita acomodar el punto decimal.

In [65]:
datos.loc[datos['Promedio'].between(10, 100, 'right'), 'Promedio'] = datos['Promedio'] / 10
datos.loc[datos['Promedio'] > 100, 'Promedio'] = datos['Promedio'] / 100
mostrar_unicos('Promedio').sort_values(ascending=False)

37    10.00
36     9.90
35     9.80
34     9.70
33     9.60
32     9.58
31     9.50
30     9.40
29     9.30
28     9.20
27     9.10
26     9.00
25     8.90
24     8.86
23     8.80
22     8.70
21     8.60
20     8.50
19     8.40
18     8.30
17     8.20
16     8.10
15     8.00
14     7.90
13     7.80
12     7.70
11     7.60
10     7.50
9      7.40
8      7.30
7      7.20
6      7.10
5      7.00
4      6.80
3      6.30
2      6.00
1      4.00
0      0.00
38      NaN
Name: Promedio, dtype: float64

In [66]:
# TODO: revisar esta propuesta:
# PROPUESTA --> Sustituimos los valores faltantes con el promedio de los datos o con un valor constante como 0.
datos['Promedio'] = datos['Promedio'].fillna(datos['Promedio'].mean())

## telefonocelular

In [67]:
mostrar_unicos('telefonocelular')

0      N
1      Y
2    NaN
Name: telefonocelular, dtype: object

Se sustituye los valores faltantes de la columna con "no" (cuya clav es `N`).

In [68]:
llenar_vacios('telefonocelular', 'B2')

/Users/erickavendanogarcia/Downloads/servicio_social/encuestas/auxiliar_4.py:143: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[columna].fillna(valor, inplace=True)


## carrera84

Se refiere a cuándo entró a la universidad: antes o después de la pandemia.

In [69]:
llenar_vacios('carrera84', 'B2')

## saludantesde

Se pregunta cómo se considera que era la salud antes de la carrera

In [70]:
llenar_vacios('saludantesde', 'B2')

## Salud en la carrera

### carrerasinpandemia

Salud en la carrera antes de la pandemia

In [71]:
# Cuando un valor es vacío es porque la persona no estudió en la universidad antes de la pandemia
llenar_vacios('carrerasinpandemia', 'B1')

### carrerapandemia

Salud en la carrera durante la pandemia

In [72]:
# Como se entrevista estudiantes, todos los estudiantes estudiaron durante la pandemia
llenar_vacios('carrerapandemia', 'B2')

## Condición física

- `condicionantes`: cómo se percibe la condición física antes de entrar a la carrera.
- `condicionpandemia`: cómo se percibe la condición física en la carrera con pandemia.
- `condicionsinpandemia`: cómo se percibe la condición física en la carrera sin pandemia.

### condicionantes

In [73]:
mostrar_unicos('condicionantes')

0     A2
1     A3
2     A4
3     A5
4     A6
5    NaN
Name: condicionantes, dtype: object

In [74]:
# Cuando un valor es vacío es porque la persona no recuerda
llenar_vacios('condicionantes', 'B2')

In [75]:
datos['condicionpandemia'] = datos['condicionpandemia'].fillna('A0')
llenar_vacios('condicionpandemia', 'B1')


### condicionpandemia

In [76]:
mostrar_unicos('condicionpandemia')

0    A0
1    A2
2    A3
3    A4
4    A5
5    A6
Name: condicionpandemia, dtype: object

In [77]:
# Cuando un valor es vacío es porque la persona no recuerda
llenar_vacios('condicionpandemia', 'B2')

### condicionsinpandemia

In [78]:
mostrar_unicos('condicionsinpandemia')

0     A2
1     A3
2     A4
3     A5
4     A6
5    NaN
Name: condicionsinpandemia, dtype: object

In [79]:
# Cuando un valor es vacío es porque la persona no asistió a la universidad antes de la pandemia
llenar_vacios('condicionsinpandemia', 'B1')

## Estrés

- `estresantes2`: nivel de estrés antes de la carrera.
- `estressinpandemia`: nivel de estrés en la carrera antes de la pandemia.
- `estresactual`: nivel de estrés en la carrera durante la pandemia.


### estresantes2

In [80]:
mostrar_unicos('estresantes2')

0    A2
1    A3
2    A4
3    A5
4    A6
5    A7
Name: estresantes2, dtype: object

### estressinpandemia

In [81]:
mostrar_unicos('estressinpandemia')

0     A2
1     A3
2     A4
3     A5
4     A6
5    NaN
Name: estressinpandemia, dtype: object

In [82]:
# Cuando un valor es vacío es porque la persona no estaba en la universidad antes de la pandemia
llenar_vacios('estressinpandemia', 'B1')

### estresactual

In [83]:
mostrar_unicos('estresactual')

0     A2
1     A3
2     A4
3     A5
4     A6
5     A7
6    NaN
Name: estresactual, dtype: object

## Peso

- `pesoaantes`:  cuál es el peso que se cree que se tenía antes de entrar a la carrera.
- `pesosinpandemia` y `pesoconpandemia`: el peso que se cree que se tiene durante la carrera pero en las condiciones sin y con pandemia.
- `pesoactual`: peso actual del paciente (no percepciones).
- `cambiopeso`: cambio de peso debido a la pandemia.
- `cambiodepesoentrarca`: cambio de peso por entrar a la carrera.
- `pesoultimavez`: peso de la última vez que se pesó.
- `cadacuandopesas`: cada cuándo se pesa el encuestado.
- `accionespeso`: acciones que se desea tomar respecto al peso.


### pesoantes

In [84]:
mostrar_unicos('pesoaantes')

0    A2
1    A3
2    A4
3    A5
4    A6
5    A7
6    A8
Name: pesoaantes, dtype: object

### pesosinpandemia

In [85]:
mostrar_unicos('pesosinpandemia')

0     A3
1     A4
2     A5
3     A6
4    NaN
Name: pesosinpandemia, dtype: object

In [86]:
# Cuando un valor es vacío es porque la persona no estaba en la universidad antes de la pandemia
llenar_vacios('pesosinpandemia', 'B1')

### pesoconpandemia

In [87]:
mostrar_unicos('pesoconpandemia')

0     A2
1     A3
2     A4
3     A5
4     A6
5     A7
6     A8
7    NaN
Name: pesoconpandemia, dtype: object

### pesoactual

Se observa muchos valores irregulares

In [88]:
mostrar_unicos('pesoactual').sort_values(ascending=False)

146    No se sabe
145         NO SE
144          98.4
143            98
142          97.5
          ...    
3             113
2             112
1           108.5
0             108
147           NaN
Name: pesoactual, Length: 148, dtype: object

In [89]:
contar_faltantes('pesoactual')

30

Como hay varios faltantes, primero se trabajará con los demás valores.

In [90]:
llenos = datos['pesoactual'][datos['pesoactual'].notna()]
sin_numeros = llenos[~llenos.str.contains(r'\d+.\d+')].drop_duplicates()
problematicos = sin_numeros[sin_numeros.str.contains(r'[A-Za-z\Wñ]+')].drop_duplicates()

In [91]:
# Si no hay números, entonces se reemplaza por nan
valores_por_quitar = problematicos[~problematicos.str.contains(r'\d+')].unique()
datos.loc[datos['pesoactual'].isin(valores_por_quitar), 'pesoactual'] = np.nan

In [92]:
# Si hay números, entonces se extrae los dígitos
valores_por_limpiar = problematicos[problematicos.str.contains(r'\d+')].drop_duplicates()
valores_limpios = valores_por_limpiar.str.replace(r'[A-Za-z]+\.?', '', regex=True)
cambios = dict(zip(valores_por_limpiar, valores_limpios))
datos['pesoactual'] = datos['pesoactual'].replace(cambios)

In [93]:
# Los rangos son de la forma 14-16, la estrategia es quedarse con el primer número
def quitar_rango(texto):
    if texto is np.nan:
        return np.nan
    resultado = re.findall(r'(\d+)-\d+', texto)
    return resultado[0] if len(resultado) > 0 else texto


datos['pesoactual'] = datos['pesoactual'].apply(quitar_rango).astype(float)
datos['pesoactual'].describe()

count    688.000000
mean      62.710756
std       13.613920
min       37.000000
25%       53.000000
50%       60.000000
75%       69.000000
max      160.000000
Name: pesoactual, dtype: float64

Ahora se imputa los valores faltantes con la mediana

In [94]:
datos['pesoactual'] = datos['pesoactual'].fillna(datos['pesoactual'].median())

### cambiodepeso

In [95]:
mostrar_unicos('cambiodepeso')

0     A1
1     A2
2     A3
3     A4
4    NaN
Name: cambiodepeso, dtype: object

In [96]:
llenar_vacios('cambiodepeso', 'B2')

/Users/erickavendanogarcia/Downloads/servicio_social/encuestas/auxiliar_4.py:143: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[columna].fillna(valor, inplace=True)


### cambiodepesoentrarca

In [97]:
mostrar_unicos('cambiodepesoentrarca')

0     A1
1     A2
2     A3
3     A4
4    NaN
Name: cambiodepesoentrarca, dtype: object

In [98]:
llenar_vacios('cambiodepesoentrarca', 'B2')

### pesoultimavez

In [99]:
mostrar_unicos('pesoultimavez')

0     A2
1     A3
2     A4
3     A5
4     A6
5     A7
6    NaN
Name: pesoultimavez, dtype: object

In [100]:
llenar_vacios('pesoultimavez', 'B2')

### cadacuandopesas

In [101]:
mostrar_unicos('cadacuandopesas')

0    -oth-
1      A10
2       A2
3       A3
4       A4
5       A5
6       A6
7       A7
8       A8
9       A9
Name: cadacuandopesas, dtype: object

In [102]:
llenar_respuesta_libre('cadacuandopesas')
mapear_codigos('cadacuandopesas')
mostrar_unicos('cadacuandopesas')
#TODO: Revisar si se crea código para médico

/Users/erickavendanogarcia/Downloads/servicio_social/encuestas/auxiliar_4.py:99: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[columna].replace(mapeo[columna], inplace=True)


0                                   1 vez a la semana_7
1                                        1 vez al mes_5
2                                       1 vez por año_2
3                                    2-3 veces al mes_6
4                                2-3 veces por semana_8
5                                       CADA 4-5 MESES 
6                             CADA QUE ASISTE AL MEDICO
7                               CADA VEZ QUE ME ENFERMO
8                                           CITA MEDICA
9                                     CUANDO ME ENFERMO
10    CUANDO TENGO LA OPORTUNIDAD, PERO NO ES FRECUENTE
11                                     Cada 2-3 meses_4
12                                       Cada 6 meses_3
13                               Cada que voy al doctor
14                                             Diario_9
15                                  EN VISITA AL MEDICO
16                                               MEDICO
17                                         No me

In [103]:
unificar_observaciones('cadacuandopesas', r'm[eé]dic[oa]|doctor', 'medico')
mostrar_unicos('cadacuandopesas')

0                                   1 vez a la semana_7
1                                        1 vez al mes_5
2                                       1 vez por año_2
3                                    2-3 veces al mes_6
4                                2-3 veces por semana_8
5                                       CADA 4-5 MESES 
6                               CADA VEZ QUE ME ENFERMO
7                                     CUANDO ME ENFERMO
8     CUANDO TENGO LA OPORTUNIDAD, PERO NO ES FRECUENTE
9                                      Cada 2-3 meses_4
10                                       Cada 6 meses_3
11                                             Diario_9
12                                         No me peso_1
13                                             RARA VEZ
14                                        cuando quiero
15                                      esporadicamente
16                                               medico
Name: cadacuandopesas, dtype: object

### accionespeso

In [104]:
mostrar_unicos('accionespeso')

0     A2
1     A3
2     A4
3     A5
4     A6
5    NaN
Name: accionespeso, dtype: object

In [105]:
llenar_vacios('accionespeso', 'B2')

/Users/erickavendanogarcia/Downloads/servicio_social/encuestas/auxiliar_4.py:143: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[columna].fillna(valor, inplace=True)


## haconsultadoDr

Las siguientes preguntas son iguales: ¿Has consultado o recibio tratamiento de un profesional de la salud al respecto?, lo que cambia es el momento:
- `haconsultadoDr[SQ002]`: nunca
- `haconsultadoDr[SQ003]`: pasado
- `haconsultadoDr[SQ004]`: actual
- `haconsultadoDr[SQ005]`: no sé


In [106]:
for i in range(2, 6):
    columna = f'haconsultadoDr[SQ00{i}]'
    mostrar_unicos(columna)
    contar_faltantes(columna)
    llenar_vacios(columna, 'B2')

/Users/erickavendanogarcia/Downloads/servicio_social/encuestas/auxiliar_4.py:143: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'B2' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df[columna].fillna(valor, inplace=True)


## altura

Se medirá la estatura en metros

In [107]:
mostrar_unicos('altura')

0       1.41
1       1.45
2       1.46
3       1.47
4       1.48
       ...  
77    182.00
78    183.00
79    184.00
80    192.00
81       NaN
Name: altura, Length: 82, dtype: float64

In [108]:
datos.loc[datos['altura'].between(3, 200, inclusive='right'), 'altura'] = datos['altura'] / 100
mostrar_unicos('altura')

0     1.41
1     1.45
2     1.46
3     1.47
4     1.48
5     1.49
6     1.50
7     1.51
8     1.52
9     1.53
10    1.54
11    1.55
12    1.56
13    1.57
14    1.58
15    1.59
16    1.60
17    1.61
18    1.62
19    1.63
20    1.64
21    1.65
22    1.66
23    1.67
24    1.68
25    1.69
26    1.70
27    1.71
28    1.72
29    1.73
30    1.74
31    1.75
32    1.76
33    1.77
34    1.78
35    1.79
36    1.80
37    1.81
38    1.82
39    1.83
40    1.84
41    1.85
42    1.86
43    1.87
44    1.92
45    2.00
46     NaN
Name: altura, dtype: float64

Se imputa con la mediana

In [109]:
datos['altura'] = datos['altura'].fillna(datos['altura'].median())

## IMC

- `IMC`: ¿el encuestado conoce su IMC?
- `IMC2`: si lo conoce, se indica el valor


In [110]:
mostrar_unicos('IMC')

0    -oth-
1       A2
2       A3
3       A4
4      NaN
Name: IMC, dtype: object

In [111]:
mapeo_respuestas['IMC']

{'A2': 'Sí_1', 'A3': 'No_2', 'A4': 'No se que es Índice de Masa Corporal_3'}

Muchas personas anotan su IMC en la columna `IMC[other]`, se extrae el valor numérico:

In [112]:
mostrar_unicos('IMC[other]')

0          18.5
1         19.49
2         20.78
3          20.9
4          22.7
5          23.2
6          24.5
7          24.6
8          28.4
9          36.1
10    APROX. 21
11          NaN
Name: IMC[other], dtype: object

Primero se limpia la columna:

In [113]:
datos['IMC[other]'] = (
    datos['IMC[other]']
    .str.replace(r'APROX\.\s+', '', regex=True)
    .astype('float')
)
mostrar_unicos('IMC[other]')

0     18.50
1     19.49
2     20.78
3     20.90
4     21.00
5     22.70
6     23.20
7     24.50
8     24.60
9     28.40
10    36.10
11      NaN
Name: IMC[other], dtype: float64

Se combina los valores numéricos con los de la columna `IMC2`:

In [114]:
mostrar_unicos('IMC2')

0        8.0
1       14.0
2       17.6
3       17.9
4       18.0
       ...  
64      35.9
65      38.0
66      44.0
67    1959.0
68       NaN
Name: IMC2, Length: 69, dtype: float64

In [115]:
datos['IMC2'] = datos['IMC2'].combine_first(datos['IMC[other]'])
datos.drop(columns='IMC[other]', inplace=True)
mostrar_unicos('IMC2')

0        8.0
1       14.0
2       17.6
3       17.9
4       18.0
       ...  
71      36.1
72      38.0
73      44.0
74    1959.0
75       NaN
Name: IMC2, Length: 76, dtype: float64

Ahora se corrige los valores atípicos

In [116]:
datos.loc[datos['IMC2'] > 100, 'IMC2'] = datos['IMC2'] / 10
datos.loc[datos['IMC2'] > 100, 'IMC2'] = datos['IMC2'] / 10
datos['IMC2'] = datos['IMC2'].round(2)
mostrar_unicos('IMC2')

0      8.0
1     14.0
2     17.6
3     17.9
4     18.0
      ... 
71    35.9
72    36.1
73    38.0
74    44.0
75     NaN
Name: IMC2, Length: 76, dtype: float64

Entonces en algunos casos "other" se puede cambiar por "A2" y en otros por "A3":

In [117]:
filtro_otros = (datos['IMC'] == '-oth-') | datos['IMC'].isna()
datos.loc[filtro_otros & datos['IMC2'].notna(), 'IMC'] = 'A2'
datos.loc[filtro_otros & datos['IMC2'].isna(), 'IMC'] = 'A3'

In [118]:
datos.loc[datos['IMC2'].notna(), 'IMC'] = 'A2'
datos.loc[(datos['IMC'] == '-oth-') & (datos['IMC2'].notna()), 'IMC'] = 'A2'

In [119]:
llenar_vacios('IMC', '901')
mostrar_unicos('IMC')

/Users/erickavendanogarcia/Downloads/servicio_social/encuestas/auxiliar_4.py:143: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[columna].fillna(valor, inplace=True)


0    A2
1    A3
2    A4
Name: IMC, dtype: object

In [120]:
b = datos['IMC'] == 'A3'
a = datos['IMC2'].isna()
(b & a).sum()

601

In [121]:
no_conoce_imc = datos['IMC'] == 'A3'
sin_imc = datos['IMC2'].isna() | (datos['IMC2'] == 0)
datos.loc[no_conoce_imc & sin_imc, 'IMC2'] = 901
mostrar_unicos('IMC2')

0       8.0
1      14.0
2      17.6
3      17.9
4      18.0
      ...  
72     36.1
73     38.0
74     44.0
75    901.0
76      NaN
Name: IMC2, Length: 77, dtype: float64

In [122]:
imc_calculado = (datos['pesoactual'] / datos['altura'] ** 2).round(2)
datos.insert(datos.columns.get_loc('IMC2') + 1, 'IMC3', imc_calculado)

In [123]:
datos

,id,submitdate,lastpage,startlanguage,seed,startdate,datestamp,encuestador,conset,q35,...,y65[SQ002],y65[SQ003],y65[SQ004],y65[SQ005],y65[SQ006],y65[SQ007],y65[SQ008],y65[SQ009],y65[SQ010],sdfds3
0,67,2021-03-27 13:32:28,5,es-MX,909191565,2021-03-27 13:03:10,2021-03-27 13:32:28,A13,Y,Y,...,NECESARIO,IMPORTANTE,SOCIAL,PROTECCION,AUTOCUIDADO,FAMILIA,SALUD,COVID,HOSPITAL,10.0
1,68,2021-03-27 19:38:43,5,es-MX,99489830,2021-03-27 19:07:21,2021-03-27 19:38:43,A13,Y,Y,...,ESTIMULACION,NEURONAL,CONEXION,LONGEVIDAD,FORTALEZA,NaN,NaN,NaN,NaN,6.0
2,69,2021-03-29 11:47:46,5,es-MX,1592450376,2021-03-29 11:04:42,2021-03-29 11:47:46,A07,Y,Y,...,HOSPITAL,ENFERMEDAD,COVID,PANDEMIA,NECESARIO,UTIL,INCOMODO,NaN,NaN,8.0
3,70,2021-03-29 12:37:20,5,es-MX,762883078,2021-03-29 12:04:01,2021-03-29 12:37:20,A04,Y,Y,...,virus,pandemia,situacion,global,estres,ansiedad,muertes,NaN,NaN,8.0
4,71,2021-03-29 14:33:34,5,es-MX,1746037044,2021-03-29 14:00:48,2021-03-29 14:33:34,A03,Y,Y,...,PANDEMIA,PREVENCION,ENFERMEDAD,CUARENTENA,NECESARIO,SALUD,INCOMODO,NaN,NaN,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
718,851,2022-10-29 17:28:54,5,es-MX,584177600,2022-10-29 17:07:05,2022-10-29 17:28:54,A05,Y,Y,...,MALA ALIMENTACION,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0
719,852,2022-10-29 18:20:26,5,es-MX,921555126,2022-10-29 18:02:41,2022-10-29 18:20:26,A05,Y,Y,...,CIENCIA,TERMINO,AUTOCUIDADO,NaN,NaN,NaN,NaN,NaN,NaN,4.0
720,853,2022-10-29 20:23:28,5,es-MX,114360708,2022-10-29 20:02:20,2022-10-29 20:23:28,A05,Y,Y,...,DISCIPLINA,SALUD,BUENO,CONSCIENCIA,RIGUROSO,TIEMPO,NaN,NaN,NaN,7.0
721,854,2022-10-31 19:02:54,5,es-MX,2117179292,2022-10-31 18:32:54,2022-10-31 19:02:54,A11,Y,Y,...,bienestar,constancia,cuerpo,sudor,autoestima,reto,atención,diversión,paz,10.0


## Enfermedades (dxcronica)

Se pregunta sobre si el encuestado ha sido diganósticado con alguna enfermedad y en caso de que sí, a qué edad. Entonces, para cada enfermedad hay dos columnas.
Los códigos de cada enfermedad son:
- dfb: Sobrepeso u obesidad
- dsfh:  Diabetes
- shth: Prediabetes
- cxbh: Síndrome metabólico
- syth:  Hipertensión
- svdg: Colesterol alto
- ty: Triglicéridos altos
- fdsty:  Cáncer
- fg:  Algún problema cardiovascular
- 4565: Algún problema respiratorio
- 36e4: COVID19
- 7659ty: Alcoholismo
- 345q: Tabaquismo
- 867: Trastorno de la alimentación (bulimia, anorexia)
- 35: Trastorno de atracón
- 76: Ansiedad
- 987: Depresión
- 2345: Trastorno obsesivo compulsivo
- 234: Otra adicción
- other: Otro

La primer columna de cada enfermedad es de la forma `dx[<enfermedad>]`.
La segunda columna de cada enfermedad es de la forma `dx[<enfermedad>comment]`.


Primero se extrae las columnas que contienen información sobre las enfermedades.

In [124]:
diccionario.index = diccionario.index.astype(str)

filtro_todas = diccionario.index.str.contains(r'dxcronica\[\w+\]', regex=True)
filtro_edad = diccionario.index.str.contains(r'dxcronica\[\w+comment\]', regex=True)

#filtro_todas = pd.Series(filtro_todas, dtype=bool)
#filtro_edad = pd.Series(filtro_edad, dtype=bool)

filtro_pregunta = filtro_todas & ~filtro_edad

In [125]:
columnas_enfermedades = diccionario.index[filtro_todas]
columnas_enfermedades

Index(['dxcronica[dfb]', 'dxcronica[dfbcomment]', 'dxcronica[dsfh]',
       'dxcronica[dsfhcomment]', 'dxcronica[shth]', 'dxcronica[shthcomment]',
       'dxcronica[cxbh]', 'dxcronica[cxbhcomment]', 'dxcronica[syth]',
       'dxcronica[sythcomment]', 'dxcronica[svdg]', 'dxcronica[svdgcomment]',
       'dxcronica[ty]', 'dxcronica[tycomment]', 'dxcronica[fdsty]',
       'dxcronica[fdstycomment]', 'dxcronica[fg]', 'dxcronica[fgcomment]',
       'dxcronica[4565]', 'dxcronica[4565comment]', 'dxcronica[36e4]',
       'dxcronica[36e4comment]', 'dxcronica[7659ty]',
       'dxcronica[7659tycomment]', 'dxcronica[345q]', 'dxcronica[345qcomment]',
       'dxcronica[867]', 'dxcronica[867comment]', 'dxcronica[35]',
       'dxcronica[35comment]', 'dxcronica[76]', 'dxcronica[76comment]',
       'dxcronica[987]', 'dxcronica[987comment]', 'dxcronica[2345]',
       'dxcronica[2345comment]', 'dxcronica[234]', 'dxcronica[234comment]',
       'dxcronica[other]', 'dxcronica[othercomment]'],
      dtype='obje

In [126]:
columnas_edades = diccionario.index[filtro_edad]
columnas_edades

Index(['dxcronica[dfbcomment]', 'dxcronica[dsfhcomment]',
       'dxcronica[shthcomment]', 'dxcronica[cxbhcomment]',
       'dxcronica[sythcomment]', 'dxcronica[svdgcomment]',
       'dxcronica[tycomment]', 'dxcronica[fdstycomment]',
       'dxcronica[fgcomment]', 'dxcronica[4565comment]',
       'dxcronica[36e4comment]', 'dxcronica[7659tycomment]',
       'dxcronica[345qcomment]', 'dxcronica[867comment]',
       'dxcronica[35comment]', 'dxcronica[76comment]', 'dxcronica[987comment]',
       'dxcronica[2345comment]', 'dxcronica[234comment]',
       'dxcronica[othercomment]'],
      dtype='object', name='code')

In [127]:
columnas_preguntas = diccionario.index[filtro_pregunta]
columnas_preguntas

Index(['dxcronica[dfb]', 'dxcronica[dsfh]', 'dxcronica[shth]',
       'dxcronica[cxbh]', 'dxcronica[syth]', 'dxcronica[svdg]',
       'dxcronica[ty]', 'dxcronica[fdsty]', 'dxcronica[fg]', 'dxcronica[4565]',
       'dxcronica[36e4]', 'dxcronica[7659ty]', 'dxcronica[345q]',
       'dxcronica[867]', 'dxcronica[35]', 'dxcronica[76]', 'dxcronica[987]',
       'dxcronica[2345]', 'dxcronica[234]', 'dxcronica[other]'],
      dtype='object', name='code')

### Preguntas

In [128]:
datos[columnas_preguntas].isna().mean()

dxcronica[dfb]       0.722992
dxcronica[dsfh]      0.997230
dxcronica[shth]      0.981994
dxcronica[cxbh]      0.995845
dxcronica[syth]      0.991690
dxcronica[svdg]      0.963989
dxcronica[ty]        0.955679
dxcronica[fdsty]     0.991690
dxcronica[fg]        0.983380
dxcronica[4565]      0.908587
dxcronica[36e4]      0.792244
dxcronica[7659ty]    0.991690
dxcronica[345q]      0.977839
dxcronica[867]       0.957064
dxcronica[35]        0.975069
dxcronica[76]        0.771468
dxcronica[987]       0.807479
dxcronica[2345]      0.980609
dxcronica[234]       0.995845
dxcronica[other]     0.855956
dtype: float64

La mayoría de las columnas de pregunta solo tienen dos tipos de valores, así que los faltantes se llenan con `B2` ya que no dependen de la pandemia:

In [129]:
for col in columnas_preguntas:
    print(mostrar_unicos(col))

0      Y
1    NaN
Name: dxcronica[dfb], dtype: object
0      Y
1    NaN
Name: dxcronica[dsfh], dtype: object
0      Y
1    NaN
Name: dxcronica[shth], dtype: object
0      Y
1    NaN
Name: dxcronica[cxbh], dtype: object
0      Y
1    NaN
Name: dxcronica[syth], dtype: object
0      Y
1    NaN
Name: dxcronica[svdg], dtype: object
0      Y
1    NaN
Name: dxcronica[ty], dtype: object
0      Y
1    NaN
Name: dxcronica[fdsty], dtype: object
0      Y
1    NaN
Name: dxcronica[fg], dtype: object
0      Y
1    NaN
Name: dxcronica[4565], dtype: object
0      Y
1    NaN
Name: dxcronica[36e4], dtype: object
0      Y
1    NaN
Name: dxcronica[7659ty], dtype: object
0      Y
1    NaN
Name: dxcronica[345q], dtype: object
0      Y
1    NaN
Name: dxcronica[867], dtype: object
0      Y
1    NaN
Name: dxcronica[35], dtype: object
0      Y
1    NaN
Name: dxcronica[76], dtype: object
0      Y
1    NaN
Name: dxcronica[987], dtype: object
0      Y
1    NaN
Name: dxcronica[2345], dtype: object
0      Y
1    NaN


In [130]:
for col in columnas_preguntas:
    llenar_vacios(col, 'B2')

/Users/erickavendanogarcia/Downloads/servicio_social/encuestas/auxiliar_4.py:143: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[columna].fillna(valor, inplace=True)


### Edades

Las respuestas son muy irregulares, a pesar de que se esperan valores numéricos (salvo en la pregunta "otro"):

In [131]:
for col in columnas_edades:
    print(mostrar_unicos(col))

0               10
1            10 11
2               11
3               12
4               13
5               14
6               15
7               16
8               17
9               18
10         18 años
11              19
12               2
13              20
14              21
15              22
16              23
17              26
18              28
19               3
20              30
21              31
22              32
23              33
24              34
25              35
26              36
27               4
28               5
29              50
30               6
31               7
32               8
33               9
34    sobrepeso 20
35             NaN
Name: dxcronica[dfbcomment], dtype: object
0    17.0
1    44.0
2     NaN
Name: dxcronica[dsfhcomment], dtype: float64
0     10-12
1        12
2        15
3        17
4        18
5        19
6        21
7        22
8        23
9        30
10       37
11      NaN
Name: dxcronica[shthcomment], dtype: object
0    20.0


En muchas ocasiones se menciona la palabra "años", así que se puede quitar:

In [132]:
columnas_texto = datos[columnas_edades].select_dtypes(include='object').columns
columnas_texto

Index(['dxcronica[dfbcomment]', 'dxcronica[shthcomment]',
       'dxcronica[svdgcomment]', 'dxcronica[tycomment]',
       'dxcronica[fgcomment]', 'dxcronica[4565comment]',
       'dxcronica[36e4comment]', 'dxcronica[7659tycomment]',
       'dxcronica[867comment]', 'dxcronica[35comment]', 'dxcronica[76comment]',
       'dxcronica[987comment]', 'dxcronica[234comment]',
       'dxcronica[othercomment]'],
      dtype='object')

In [133]:
for col in columnas_texto:
    datos[col] = datos[col].str.replace('\s*años\s*', '', regex=True)

<>:2: SyntaxWarning: invalid escape sequence '\s'
<>:2: SyntaxWarning: invalid escape sequence '\s'
/var/folders/hb/c3zh8bns0mx4nw98_6k7rx8m0000gn/T/ipykernel_4464/2207088439.py:2: SyntaxWarning: invalid escape sequence '\s'
  datos[col] = datos[col].str.replace('\s*años\s*', '', regex=True)


In [134]:
for col in columnas_edades:
    print(mostrar_unicos(col))

0               10
1            10 11
2               11
3               12
4               13
5               14
6               15
7               16
8               17
9               18
10              19
11               2
12              20
13              21
14              22
15              23
16              26
17              28
18               3
19              30
20              31
21              32
22              33
23              34
24              35
25              36
26               4
27               5
28              50
29               6
30               7
31               8
32               9
33    sobrepeso 20
34             NaN
Name: dxcronica[dfbcomment], dtype: object
0    17.0
1    44.0
2     NaN
Name: dxcronica[dsfhcomment], dtype: float64
0     10-12
1        12
2        15
3        17
4        18
5        19
6        21
7        22
8        23
9        30
10       37
11      NaN
Name: dxcronica[shthcomment], dtype: object
0    20.0
1    25.0
2    37.0

In [135]:
unificar_observaciones('dxcronica[fgcomment]', r'naci', '0')
unificar_observaciones('dxcronica[4565comment]', r'\s*asma\*', '0')
unificar_observaciones('dxcronica[othercomment]', r'al año de nacer', '1')
unificar_observaciones('dxcronica[othercomment]', r'nacer', '0')

In [136]:
for col in columnas_edades:
    print(mostrar_unicos(col))

0               10
1            10 11
2               11
3               12
4               13
5               14
6               15
7               16
8               17
9               18
10              19
11               2
12              20
13              21
14              22
15              23
16              26
17              28
18               3
19              30
20              31
21              32
22              33
23              34
24              35
25              36
26               4
27               5
28              50
29               6
30               7
31               8
32               9
33    sobrepeso 20
34             NaN
Name: dxcronica[dfbcomment], dtype: object
0    17.0
1    44.0
2     NaN
Name: dxcronica[dsfhcomment], dtype: float64
0     10-12
1        12
2        15
3        17
4        18
5        19
6        21
7        22
8        23
9        30
10       37
11      NaN
Name: dxcronica[shthcomment], dtype: object
0    20.0
1    25.0
2    37.0

In [137]:
datos['dxcronica[dfbcomment]'] = datos['dxcronica[dfbcomment]'].str.replace(r' años| \d+|[a-z]+ ', '',
                                                                            regex=True).astype(float)
datos['dxcronica[dsfhcomment]'] = datos['dxcronica[dsfhcomment]'].astype(float)
datos['dxcronica[shthcomment]'] = datos['dxcronica[shthcomment]'].str.replace(r'\D', '', regex=True).astype(float)
datos['dxcronica[cxbhcomment]'] = datos['dxcronica[cxbhcomment]'].astype(float)
datos['dxcronica[sythcomment]'] = datos['dxcronica[sythcomment]'].astype(float)
datos['dxcronica[svdgcomment]'] = datos['dxcronica[svdgcomment]'].str.replace(r' años', '', regex=True).astype(float)
datos['dxcronica[tycomment]'] = datos['dxcronica[tycomment]'].str.replace(r' años', '', regex=True).astype(float)
datos['dxcronica[fdstycomment]'] = datos['dxcronica[fdstycomment]'].astype(float)
datos['dxcronica[fgcomment]'] = datos['dxcronica[fgcomment]'] \
    .str.replace(r'recién nacida|nacimiento|al nacer', '0', regex=True, flags=re.IGNORECASE) \
    .str.replace(r'al año de nacer', '1', regex=True, flags=re.IGNORECASE) \
    .str.strip().astype(float)
datos['dxcronica[4565comment]'] = datos['dxcronica[4565comment]'].str.replace(r'años|asma|meses|\s+|\[a-zA-Z]+', '',
                                                                              regex=True)\
                                                                 .str.replace(' ', '0', regex=True)
                                                                              #.str.replace('', '0', regex=True)

datos['dxcronica[4565comment]'] = datos['dxcronica[4565comment]'].astype(str).apply(lambda value: '0' if 'meses' in value and int(value.split()[0]) < 12 else str(int(value.split()[0]) // 12) if 'meses' in value else value).replace('', np.nan)


datos['dxcronica[36e4comment]'] = datos['dxcronica[36e4comment]'].str.replace(r'[A-Za-zñ\s]+', '', regex=True).astype(
    float)
datos['dxcronica[7659tycomment]'] = datos['dxcronica[7659tycomment]'].str.replace(r' a \d+', '', regex=True).astype(
    float)
datos['dxcronica[345qcomment]'] = datos['dxcronica[345qcomment]'].astype(float)
datos['dxcronica[867comment]'] = datos['dxcronica[867comment]'].str.replace(r' \d+', '', regex=True).astype(float)
datos['dxcronica[35comment]'] = datos['dxcronica[35comment]'].str.replace(r' \d+', '', regex=True).astype(float)
datos['dxcronica[76comment]'] = datos['dxcronica[76comment]'].str.replace(r' años', '', regex=True).astype(float)
datos['dxcronica[987comment]'] = datos['dxcronica[987comment]'].str.replace(r' años| \d+', '', regex=True).astype(float)
datos['dxcronica[2345comment]'] = datos['dxcronica[2345comment]'].astype(float)
datos['dxcronica[234comment]'] = datos['dxcronica[234comment]'].str.replace(r'[\D\s]+',
                                                                            '',
                                                                            regex=True).replace('', np.nan).astype(
    float)

## D1

In [138]:
mostrar_unicos('D1[SQ0022]')

0    A10
1     A2
2     A3
3     A4
4     A5
5     A6
6     A7
7     A8
8     A9
Name: D1[SQ0022], dtype: object

In [139]:
mostrar_unicos('D1[SQ002]')

0     A10
1     A11
2      A2
3      A3
4      A4
5      A5
6      A6
7      A7
8      A8
9      A9
10    NaN
Name: D1[SQ002], dtype: object

In [140]:
llenar_vacios('D1[SQ002]', 'B1')

/Users/erickavendanogarcia/Downloads/servicio_social/encuestas/auxiliar_4.py:143: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[columna].fillna(valor, inplace=True)


In [141]:
mostrar_unicos('D1[SQ003]')

0    A10
1    A11
2     A2
3     A3
4     A4
5     A5
6     A6
7     A7
8     A8
9     A9
Name: D1[SQ003], dtype: object

In [142]:
mostrar_unicos('D1[SQ004]')

0     A2
1     A3
2     A4
3     A5
4     A6
5     A7
6     A8
7    NaN
Name: D1[SQ004], dtype: object

In [143]:
llenar_vacios('D1[SQ004]', 'B2')

## se34

In [144]:
mostrar_unicos('se34')

0     A1
1     A2
2    NaN
Name: se34, dtype: object

In [145]:
llenar_vacios('se34', 'B1')

## xqtrabajas

In [146]:
for col in ('xqtrabajas[12]', 'xqtrabajas[21]', 'xqtrabajas[32]', 'xqtrabajas[45]'):
    llenar_vacios(col, 'B2')

In [147]:
datos.drop(columns='xqtrabajas[other]', inplace=True)

## focospapas y focosfuera

In [148]:
# Las columnas focospapas y focosfuera tienen que ver con la cantidad de focos que hay en la vivienda de los padres y
# la vivienda donde se vive(si es fuera de la vivienda de los padres).

# Hay valores de 100 y 500 focos, supongamos que si son arriba de 100 focos, se equivocaron por un punto decimal.
# Si no se puede hacer eso, podemos eliminar los datos
datos.loc[datos['focospapas'] > 100, 'focospapas'] = datos['focospapas'] / 10
datos.loc[datos['focospapas'] > 30, 'focospapas'] = datos['focospapas'] / 10
np.sort(datos['focospapas'].unique())


array([ 2. ,  3. ,  3.2,  3.5,  3.6,  3.8,  4. ,  4.5,  5. ,  5.4,  5.5,
        6. ,  6.5,  7. ,  8. ,  9. , 10. , 11. , 12. , 13. , 14. , 15. ,
       16. , 17. , 18. , 19. , 20. , 21. , 22. , 23. , 25. , 26. , 27. ,
       28. , 30. ,  nan])

## Fumar (q0r490)

In [149]:
mostrar_unicos('q0r490')

0    -oth-
1       A2
2       A3
3       A4
4       A5
5      NaN
Name: q0r490, dtype: object

In [150]:
mostrar_unicos('q0r490[other]')

0      NO ES HABITUAL
1         NO HABITUAL
2             SOCIAL 
3    SOLO PARA PROBAR
4     esporádicamente
5      ocasionalmente
6                 NaN
Name: q0r490[other], dtype: object

In [151]:
llenar_respuesta_libre('q0r490')
datos['q0r490'].fillna('A4', inplace=True)
mostrar_unicos('q0r490')

/var/folders/hb/c3zh8bns0mx4nw98_6k7rx8m0000gn/T/ipykernel_4464/412546662.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  datos['q0r490'].fillna('A4', inplace=True)


0                  A2
1                  A3
2                  A4
3                  A5
4      NO ES HABITUAL
5         NO HABITUAL
6             SOCIAL 
7    SOLO PARA PROBAR
8     esporádicamente
9      ocasionalmente
Name: q0r490, dtype: object

In [152]:
mapear_codigos('q0r490')
mostrar_unicos('q0r490')

0      NO ES HABITUAL
1         NO HABITUAL
2        No aplica_88
3             No sé_9
4                No_2
5             SOCIAL 
6    SOLO PARA PROBAR
7                Sí_1
8     esporádicamente
9      ocasionalmente
Name: q0r490, dtype: object

In [153]:
unificar_observaciones('q0r490', r'social|ocasionalmente', 'No_2')
mostrar_unicos('q0r490')

0      NO ES HABITUAL
1         NO HABITUAL
2        No aplica_88
3             No sé_9
4                No_2
5    SOLO PARA PROBAR
6                Sí_1
7     esporádicamente
Name: q0r490, dtype: object

In [154]:
unificar_observaciones('q0r490', r'habitual|probar|espor', 'No_1')
mostrar_unicos('q0r490')

0    No aplica_88
1         No sé_9
2            No_1
3            No_2
4            Sí_1
Name: q0r490, dtype: object

In [155]:
#mapeo_respuestas['q0r490']
datos['q0r490'] = datos['q0r490'].replace({'Sí_1': 'A2', 'No_2': 'A3', 'No sé_3': 'A4', 'No aplica_4': 'A5'})
mostrar_unicos('q0r490')

0              A2
1              A3
2    No aplica_88
3         No sé_9
4            No_1
Name: q0r490, dtype: object

## Horas ejercicio

### antesdhorasejercicio

La columna `antesdehorasejercicio` son las horas que se ejercitaba habitualmente antes de entrar a la carrera. Pero no se tiene información sobre si la pregunta es diaria o semanal. Sin embargo, son horas a la semana. La OMS considera que una persona hace ejercicio si le dedica al menos lo hace 2.5 horas a la semana.

In [156]:
mostrar_unicos('antesdhorasejercicio')

0       0.0
1       1.0
2       1.5
3       2.0
4       2.5
5       3.0
6       3.5
7       4.0
8       4.5
9       5.0
10      5.5
11      6.0
12      7.0
13      8.0
14      9.0
15     10.0
16     11.0
17     12.0
18     13.0
19     14.0
20     15.0
21     16.0
22     18.0
23     20.0
24     21.0
25     23.0
26     25.0
27     30.0
28     36.0
29     61.0
30    216.0
31      NaN
Name: antesdhorasejercicio, dtype: float64

In [157]:
contar_faltantes('antesdhorasejercicio')

190

Una semana tiene 168 horas, los valores registrados que superen ese número se consideran errores de dedo, y se acordó reportarlos como perdidos

In [158]:
datos.loc[datos['antesdhorasejercicio'] > 168, 'antesdhorasejercicio'] = np.nan

### ejerciciosinpandemia y conapndemiaejercicio

La columna `ejerciciosinpandemia` es la cantidad de ejercicio que se hacia ya estando en la carrera,
pero sin pandemia. Y la columna `conapndemiaejercicio` es lo mismo, pero con pandemia

In [159]:
mostrar_unicos('ejerciciosinpandemia')

0      0.0
1      0.5
2      1.0
3      1.5
4      2.0
5      2.5
6      3.0
7      4.0
8      5.0
9      6.0
10     7.0
11     8.0
12     9.0
13    10.0
14    11.0
15    12.0
16    13.0
17    14.0
18    15.0
19    18.0
20     NaN
Name: ejerciciosinpandemia, dtype: float64

In [160]:
mostrar_unicos('conapndemiaejercicio')

0       0.0
1       0.3
2       0.5
3       1.0
4       1.5
5       2.0
6       2.5
7       3.0
8       3.5
9       4.0
10      4.5
11      5.0
12      5.5
13      6.0
14      7.0
15      8.0
16      9.0
17     10.0
18     11.0
19     12.0
20     14.0
21     15.0
22     16.0
23     18.0
24     21.0
25     30.0
26     36.0
27    130.0
28      NaN
Name: conapndemiaejercicio, dtype: float64

Hay un valor de 130, pero como no supera a 168 no se le modificará.

## q0r1004

Se le pregunta al encuestado si toma o ha tomado de manera habitual

In [161]:
mostrar_unicos('q0r1004')

0    -oth-
1      A01
2      A02
3      A03
4      NaN
Name: q0r1004, dtype: object

In [162]:
llenar_respuesta_libre('q0r1004')
mapear_codigos('q0r1004')
mostrar_unicos('q0r1004')

/Users/erickavendanogarcia/Downloads/servicio_social/encuestas/auxiliar_4.py:99: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[columna].replace(mapeo[columna], inplace=True)


0                 CADA DO SEMANAS
1                 ESPORADICAMENTE
2             MUY ESPORADICAMENTE
3                  NO ES HABITUAL
4     NO TAN HABITUAL A LA SEMANA
5                         No sé_9
6                            No_2
7                       OCASIONAL
8                  OCASIONALMENTE
9                          SOCIAL
10                        SOCIAL 
11                    SOCIALMENTE
12                SOLO EN FIESTAS
13                         Social
14                           Sí_1
15           UNO VEZ AL AÑO O DOS
16                esporádicamente
17               esporádicamente 
18                 ocasionalmente
19                         social
20                        social 
21                            NaN
Name: q0r1004, dtype: object

In [163]:
llenar_vacios('q0r1004', 'No sé_3')
unificar_observaciones('q0r1004', r'ocasional|habitual|social|espor|año', 'No_2')
unificar_observaciones('q0r1004', r'cada|fiesta', 'Sí_1')
datos['q0r1004'].replace({'Sí_1': 'A01', 'No_2': 'A02', 'No sé_3': 'A03'}, inplace=True)
mostrar_unicos('q0r1004')

/var/folders/hb/c3zh8bns0mx4nw98_6k7rx8m0000gn/T/ipykernel_4464/2156195268.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  datos['q0r1004'].replace({'Sí_1': 'A01', 'No_2': 'A02', 'No sé_3': 'A03'}, inplace=True)


0        A01
1        A02
2        A03
3    No sé_9
Name: q0r1004, dtype: object

In [164]:
mapeo_respuestas['q0r1004']

{'A01': 'Sí_1', 'A02': 'No_2', 'A03': 'No sé_9', 'A04': 'No aplica_88'}

## Comida

- sinpandemiacomid: cuánto comía durante la carrera, sin pandemia
- conpandemiacomida: cuánto comía durante la carrera, con pandemia
- antesdecomidas: cuánto comía antes de entrar a la carrera


In [165]:
mostrar_unicos('sinpandemiacomid')

0    -oth-
1       A2
2       A3
3       A4
4       A5
5       A6
6      NaN
Name: sinpandemiacomid, dtype: object

In [166]:
llenar_respuesta_libre('sinpandemiacomid')
mapear_codigos('sinpandemiacomid')
mostrar_unicos('sinpandemiacomid')

/Users/erickavendanogarcia/Downloads/servicio_social/encuestas/auxiliar_4.py:99: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[columna].replace(mapeo[columna], inplace=True)


0                   Lo recomendado_3
1          Menos de lo recomendado_2
2    Mucho Menos de lo recomendado_1
3      Mucho más de lo recomendado_5
4            Más de lo recomendado_4
5            POR TEMPORADAS CAMBIABA
6                                NaN
Name: sinpandemiacomid, dtype: object

In [167]:
unificar_observaciones('sinpandemiacomid', r'temporada', 'Menos de lo recomendado_2')
llenar_vacios('sinpandemiacomid', 'Menos de lo recomendado_2')
mostrar_unicos('sinpandemiacomid')

/Users/erickavendanogarcia/Downloads/servicio_social/encuestas/auxiliar_4.py:143: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[columna].fillna(valor, inplace=True)


0                   Lo recomendado_3
1          Menos de lo recomendado_2
2    Mucho Menos de lo recomendado_1
3      Mucho más de lo recomendado_5
4            Más de lo recomendado_4
Name: sinpandemiacomid, dtype: object

In [168]:
mapeo_inverso = {v: k for k, v in mapeo_respuestas['sinpandemiacomid'].items()}
datos['sinpandemiacomid'].replace(mapeo_inverso, inplace=True)
mostrar_unicos('sinpandemiacomid')

0    A2
1    A3
2    A4
3    A5
4    A6
Name: sinpandemiacomid, dtype: object

# Estímulos y respuestas

Las 6 palabras estímulo son:

In [169]:
estimulos = ['ejercicio', 'cubrebocas', 'vacuna', 'covid', 'dieta', 'obesidad']

## Explicación estructura

El encuestador dice cada uno de los 6 estímulos y el encuestado debe mencionar 10 palabras que le vengan a la mente.
Por esta razón, la dinámica se conforma de 6 rondas. Cada una consiste de una palabra estímulo dicha al azar y de las 10 respuestas correspondientes.
Entonces, para cada ronda, se tiene 11 columnas contiguas (la primera de las cuales corresponde a la palabra estímulo).

Las 6 columnas que contienen el estímulo de cada ronda se denominarán **columnas detonantes** y sus nombres dentro del conjunto de datos son:

In [170]:
detonantes = ['w64', 'e456', 't66', 't75', 'd52', 'q77']

In [171]:
datos[detonantes]

,w64,e456,t66,t75,d52,q77
0,EJERCICIO,COVID,DIETA,VACUNA,OBESIDAD,CUBREBOCAS
1,CUBREBOCAS,OBESIDAD,COVID,DIETA,VACUNA,EJERCICIO
2,EJERCICIO,VACUNA,DIETA,COVID,OBESIDAD,CUBREBOCAS
3,obesidad,cubrebocas,ejercicio,vacuna,dieta,covid
4,EJERCICIO,VACUNA,OBESIDAD,COVID,DIETA,CUBREBOCAS
...,...,...,...,...,...,...
718,VACUNA,EJERCICIO,CUBREBOCAS,DIETA,COVID,OBESIDAD
719,OBESIDAD,COVID,DIETA,CUBREBOCAS,EJERCICIO,VACUNA
720,CUBREBOCAS,DIETA,COVID,OBESIDAD,VACUNA,EJERCICIO
721,"salud, Covid, gente, protocolo, enfermedad, co...","salud, hábitos, comida, vida, imagen, segurida...","protección, responsabilidad, médico, virus, be...","comida, plan, compromiso, cuerpo, salud, medid...","pandemia, aprendizaje, realidad, protección, d...","salud, sudor, bienestar, cuerpo, atención, ret..."


A su vez, los nombres de las 10 columnas de respuestas de cada ronda se forman de la siguiente manera:

- Prefijo único correspondiente a cada ronda.
- Posición de mención dentro de la ronda (que va desde 1 hasta 10).

Por ejemplo, si una ronda tiene el prefijo *w4576*, entonces las columnas de respuestas de esa ronda tienen el siguiente aspecto:
- *w4576[SQ001]*
- *w4576[SQ002]*
...
- *w4576[SQ009]*.
- *w4576[SQ010]*.

Los prefijos de cada ronda son:

In [172]:
prefijos = ['w4576', 'x62', 'u75', 'rty6', 'q9', 'y65']

Por lo tanto, el nombre las columnas de respuestas es el siguiente:

In [173]:
def generar_nombres_respuestas(prefijo):
    return [f'{prefijo}[SQ{str(posicion).zfill(3)}]' for posicion in range(1, 11)]

In [174]:
columnas = [f'{prefijo}[SQ{str(i).zfill(3)}]' for prefijo in prefijos for i in range(1, 11)]
columnas

['w4576[SQ001]',
 'w4576[SQ002]',
 'w4576[SQ003]',
 'w4576[SQ004]',
 'w4576[SQ005]',
 'w4576[SQ006]',
 'w4576[SQ007]',
 'w4576[SQ008]',
 'w4576[SQ009]',
 'w4576[SQ010]',
 'x62[SQ001]',
 'x62[SQ002]',
 'x62[SQ003]',
 'x62[SQ004]',
 'x62[SQ005]',
 'x62[SQ006]',
 'x62[SQ007]',
 'x62[SQ008]',
 'x62[SQ009]',
 'x62[SQ010]',
 'u75[SQ001]',
 'u75[SQ002]',
 'u75[SQ003]',
 'u75[SQ004]',
 'u75[SQ005]',
 'u75[SQ006]',
 'u75[SQ007]',
 'u75[SQ008]',
 'u75[SQ009]',
 'u75[SQ010]',
 'rty6[SQ001]',
 'rty6[SQ002]',
 'rty6[SQ003]',
 'rty6[SQ004]',
 'rty6[SQ005]',
 'rty6[SQ006]',
 'rty6[SQ007]',
 'rty6[SQ008]',
 'rty6[SQ009]',
 'rty6[SQ010]',
 'q9[SQ001]',
 'q9[SQ002]',
 'q9[SQ003]',
 'q9[SQ004]',
 'q9[SQ005]',
 'q9[SQ006]',
 'q9[SQ007]',
 'q9[SQ008]',
 'q9[SQ009]',
 'q9[SQ010]',
 'y65[SQ001]',
 'y65[SQ002]',
 'y65[SQ003]',
 'y65[SQ004]',
 'y65[SQ005]',
 'y65[SQ006]',
 'y65[SQ007]',
 'y65[SQ008]',
 'y65[SQ009]',
 'y65[SQ010]']

Para facilitar el acceso a los datos, para cada ronda se creará un diccionario que contenga el nombre de la columna detonante y una lista con los nombres de las columnas de respuestas.

In [175]:
rondas = [{'detonante': detonantes[i], 'respuestas': generar_nombres_respuestas(prefijos[i])} for i in range(6)]
rondas

[{'detonante': 'w64',
  'respuestas': ['w4576[SQ001]',
   'w4576[SQ002]',
   'w4576[SQ003]',
   'w4576[SQ004]',
   'w4576[SQ005]',
   'w4576[SQ006]',
   'w4576[SQ007]',
   'w4576[SQ008]',
   'w4576[SQ009]',
   'w4576[SQ010]']},
 {'detonante': 'e456',
  'respuestas': ['x62[SQ001]',
   'x62[SQ002]',
   'x62[SQ003]',
   'x62[SQ004]',
   'x62[SQ005]',
   'x62[SQ006]',
   'x62[SQ007]',
   'x62[SQ008]',
   'x62[SQ009]',
   'x62[SQ010]']},
 {'detonante': 't66',
  'respuestas': ['u75[SQ001]',
   'u75[SQ002]',
   'u75[SQ003]',
   'u75[SQ004]',
   'u75[SQ005]',
   'u75[SQ006]',
   'u75[SQ007]',
   'u75[SQ008]',
   'u75[SQ009]',
   'u75[SQ010]']},
 {'detonante': 't75',
  'respuestas': ['rty6[SQ001]',
   'rty6[SQ002]',
   'rty6[SQ003]',
   'rty6[SQ004]',
   'rty6[SQ005]',
   'rty6[SQ006]',
   'rty6[SQ007]',
   'rty6[SQ008]',
   'rty6[SQ009]',
   'rty6[SQ010]']},
 {'detonante': 'd52',
  'respuestas': ['q9[SQ001]',
   'q9[SQ002]',
   'q9[SQ003]',
   'q9[SQ004]',
   'q9[SQ005]',
   'q9[SQ006]',
   'q

Para la limpieza posterior, solo se necesitará las 66 columnas correspondientes a la dinámica de las 6 rondas y la señala el `id` de cada encuestado. Por lo tanto, se trabajará con el subconjunto de datos correspondiente a estas columnas.
Al final del proceso se volverá a unir con el resto de las columnas a través del `id`.

In [176]:
columnas_rondas = []
for numero_ronda in range(len(rondas)):
    columnas_rondas.append(rondas[numero_ronda]['detonante'])
    columnas_rondas.extend(rondas[numero_ronda]['respuestas'])
datos_rondas = datos[['id'] + columnas_rondas]
datos_rondas.head()

,id,w64,w4576[SQ001],w4576[SQ002],w4576[SQ003],w4576[SQ004],w4576[SQ005],w4576[SQ006],w4576[SQ007],w4576[SQ008],...,y65[SQ001],y65[SQ002],y65[SQ003],y65[SQ004],y65[SQ005],y65[SQ006],y65[SQ007],y65[SQ008],y65[SQ009],y65[SQ010]
0,67,EJERCICIO,SALUD,BIENESTAR,IMPORTANTE,HABITUAL,NECESARIO,DIVERTIDO,SALTAR,BAILE,...,CONTROVERSIAL,NECESARIO,IMPORTANTE,SOCIAL,PROTECCION,AUTOCUIDADO,FAMILIA,SALUD,COVID,HOSPITAL
1,68,CUBREBOCAS,PROTECCION,CUIDADO,SALUD,LIMPIEZA,GERMENES,NaN,NaN,NaN,...,SALUDABLE,ESTIMULACION,NEURONAL,CONEXION,LONGEVIDAD,FORTALEZA,NaN,NaN,NaN,NaN
2,69,EJERCICIO,SALUDABLE,SALUD,BUENO,FORMA,DEPORTE,APARATOS,GIMNASIO,CORRER,...,DOCTOR,HOSPITAL,ENFERMEDAD,COVID,PANDEMIA,NECESARIO,UTIL,INCOMODO,NaN,NaN
3,70,obesidad,enfermedad,alimentacion,comida,consumo,atracon,descompensacion,ejercicio,activacion,...,enfermedad,virus,pandemia,situacion,global,estres,ansiedad,muertes,NaN,NaN
4,71,EJERCICIO,BALLET,FIGURA,PILATES,PESO,ACTIVIDAD,SALUD,CORRER,AEROBICO,...,COVID,PANDEMIA,PREVENCION,ENFERMEDAD,CUARENTENA,NECESARIO,SALUD,INCOMODO,NaN,NaN


## Limpieza detonantes

Las observaciones de las columnas detonantes deberían ser alguna de las palabras estímulo. Sin embargo, se observa que hay muchos valores diferentes:

In [177]:
# se esperaría 6 valores únicos en cada columna
datos_rondas[detonantes].nunique()

w64     130
e456    139
t66     132
t75     130
d52     135
q77     132
dtype: int64

In [178]:
datos_rondas[detonantes].sample(5, random_state=8)

,w64,e456,t66,t75,d52,q77
351,VACUNA,DIETA,COVID,EJERCICIO,CUBREBOCAS,OBESIDAD
257,EJERCICIO,VACUNA,DIETA,COVID,OBESIDAD,CUBREBOCAS
89,DIETA,COVID,OBESIDAD,VACUNA,EJERCICIO,CUBREBOCAS
185,Cubrebocas,Obesidad,Vacuna,Dieta,COVID,Ejercicio
526,COVID,EJERCICIO,VACUNA,OBESIDAD,CUBREBOCAS,DIETA


Hay pocas observaciones que tienen alguna observación faltante para las columnas detonates, así que se pueden eliminar.

In [179]:
print(f'Total de observaciones: {len(datos_rondas)}')

Total de observaciones: 722


In [180]:
datos_rondas[detonantes].isna().sum()

w64     2
e456    3
t66     2
t75     2
d52     2
q77     3
dtype: int64

In [181]:
datos_rondas.dropna(subset=detonantes, inplace=True)
print(f'Total de observaciones tras quitar valores faltantes: {len(datos_rondas)}')

Total de observaciones tras quitar valores faltantes: 717


/var/folders/hb/c3zh8bns0mx4nw98_6k7rx8m0000gn/T/ipykernel_4464/3897469548.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datos_rondas.dropna(subset=detonantes, inplace=True)


In [182]:
for col_carrera in detonantes:
    datos_rondas[col_carrera] = datos_rondas[col_carrera].str.lower().str.strip()
    # En lugar de una palabra hay una lista de palabras, por error del encuestador
    datos_rondas = datos_rondas[~datos_rondas[col_carrera].str.contains('.*\n.*\n.*\n.*')]
    # Eliminamos si hay espacios
    datos_rondas = datos_rondas[~datos_rondas[col_carrera].str.contains('\s')]
    # Pulimos algunos detalles respecto a las formas en que se pueden escribir las palabras (faltas de ortografía)
    #TODO: cambiar por distancia de cadenas
    datos_rondas[col_carrera] = datos_rondas[col_carrera].str.replace('ejer[c]?i.*', 'ejercicio',
                                                                      regex=True)
    datos_rondas[col_carrera] = datos_rondas[col_carrera].str.replace('c[ou]b[re][re]bo.*',
                                                                      'cubrebocas',
                                                                      regex=True)
    datos_rondas[col_carrera] = datos_rondas[col_carrera].str.replace('covid.*', 'covid', regex=True)
    datos_rondas[col_carrera] = datos_rondas[col_carrera].str.replace('ob[d]?esid.*', 'obesidad',
                                                                      regex=True)
    datos_rondas[col_carrera] = datos_rondas[col_carrera].str.replace('ob[d]?esid.*', 'obesidad',
                                                                      regex=True)
    datos_rondas[col_carrera] = datos_rondas[col_carrera].str.replace('ob[d]?esid.*', 'obesidad',
                                                                      regex=True)
    # Solo se mantienen las palabras que son estímulos
    datos_rondas = datos_rondas[datos_rondas[col_carrera].isin(estimulos)]


<>:6: SyntaxWarning: invalid escape sequence '\s'
<>:6: SyntaxWarning: invalid escape sequence '\s'
/var/folders/hb/c3zh8bns0mx4nw98_6k7rx8m0000gn/T/ipykernel_4464/2576850734.py:6: SyntaxWarning: invalid escape sequence '\s'
  datos_rondas = datos_rondas[~datos_rondas[col_carrera].str.contains('\s')]
/var/folders/hb/c3zh8bns0mx4nw98_6k7rx8m0000gn/T/ipykernel_4464/2576850734.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datos_rondas[col_carrera] = datos_rondas[col_carrera].str.lower().str.strip()


## Cambio de formato

Este es el paso más importante.
Hasta ahora cada columna detonante puede tener distintas palabras estímulo debido al azar en cada ronda. Para los análisis posteriores es más conveniente que cada columna detonante tenga una sola palabra estímulo.
En este caso, el nombre de la columna detonante resultante será el nombre de la palabra estímulo. Incluso se puede sustituir los sufijos de las columnas de respuestas por el nombre de la palabra estímulo.

In [183]:
for numero_ronda in range(len(rondas)):
    detonante, respuestas = rondas[numero_ronda]['detonante'], rondas[numero_ronda]['respuestas']
    for respuesta in respuestas:
        datos_rondas[respuesta] = datos_rondas[respuesta].fillna('nr')
        datos_rondas[respuesta] = datos_rondas[respuesta].str.lower().str.strip()
        datos_rondas[respuesta] = datos_rondas[respuesta].str.cat(datos_rondas[detonante], sep=' \n ')

In [184]:
datos_rondas.head()

,id,w64,w4576[SQ001],w4576[SQ002],w4576[SQ003],w4576[SQ004],w4576[SQ005],w4576[SQ006],w4576[SQ007],w4576[SQ008],...,y65[SQ001],y65[SQ002],y65[SQ003],y65[SQ004],y65[SQ005],y65[SQ006],y65[SQ007],y65[SQ008],y65[SQ009],y65[SQ010]
0,67,ejercicio,salud \n ejercicio,bienestar \n ejercicio,importante \n ejercicio,habitual \n ejercicio,necesario \n ejercicio,divertido \n ejercicio,saltar \n ejercicio,baile \n ejercicio,...,controversial \n cubrebocas,necesario \n cubrebocas,importante \n cubrebocas,social \n cubrebocas,proteccion \n cubrebocas,autocuidado \n cubrebocas,familia \n cubrebocas,salud \n cubrebocas,covid \n cubrebocas,hospital \n cubrebocas
1,68,cubrebocas,proteccion \n cubrebocas,cuidado \n cubrebocas,salud \n cubrebocas,limpieza \n cubrebocas,germenes \n cubrebocas,nr \n cubrebocas,nr \n cubrebocas,nr \n cubrebocas,...,saludable \n ejercicio,estimulacion \n ejercicio,neuronal \n ejercicio,conexion \n ejercicio,longevidad \n ejercicio,fortaleza \n ejercicio,nr \n ejercicio,nr \n ejercicio,nr \n ejercicio,nr \n ejercicio
2,69,ejercicio,saludable \n ejercicio,salud \n ejercicio,bueno \n ejercicio,forma \n ejercicio,deporte \n ejercicio,aparatos \n ejercicio,gimnasio \n ejercicio,correr \n ejercicio,...,doctor \n cubrebocas,hospital \n cubrebocas,enfermedad \n cubrebocas,covid \n cubrebocas,pandemia \n cubrebocas,necesario \n cubrebocas,util \n cubrebocas,incomodo \n cubrebocas,nr \n cubrebocas,nr \n cubrebocas
3,70,obesidad,enfermedad \n obesidad,alimentacion \n obesidad,comida \n obesidad,consumo \n obesidad,atracon \n obesidad,descompensacion \n obesidad,ejercicio \n obesidad,activacion \n obesidad,...,enfermedad \n covid,virus \n covid,pandemia \n covid,situacion \n covid,global \n covid,estres \n covid,ansiedad \n covid,muertes \n covid,nr \n covid,nr \n covid
4,71,ejercicio,ballet \n ejercicio,figura \n ejercicio,pilates \n ejercicio,peso \n ejercicio,actividad \n ejercicio,salud \n ejercicio,correr \n ejercicio,aerobico \n ejercicio,...,covid \n cubrebocas,pandemia \n cubrebocas,prevencion \n cubrebocas,enfermedad \n cubrebocas,cuarentena \n cubrebocas,necesario \n cubrebocas,salud \n cubrebocas,incomodo \n cubrebocas,nr \n cubrebocas,nr \n cubrebocas


In [185]:
datos_rondas.drop(columns=detonantes, inplace=True)

In [186]:
datos_rondas

,id,w4576[SQ001],w4576[SQ002],w4576[SQ003],w4576[SQ004],w4576[SQ005],w4576[SQ006],w4576[SQ007],w4576[SQ008],w4576[SQ009],...,y65[SQ001],y65[SQ002],y65[SQ003],y65[SQ004],y65[SQ005],y65[SQ006],y65[SQ007],y65[SQ008],y65[SQ009],y65[SQ010]
0,67,salud \n ejercicio,bienestar \n ejercicio,importante \n ejercicio,habitual \n ejercicio,necesario \n ejercicio,divertido \n ejercicio,saltar \n ejercicio,baile \n ejercicio,activada \n ejercicio,...,controversial \n cubrebocas,necesario \n cubrebocas,importante \n cubrebocas,social \n cubrebocas,proteccion \n cubrebocas,autocuidado \n cubrebocas,familia \n cubrebocas,salud \n cubrebocas,covid \n cubrebocas,hospital \n cubrebocas
1,68,proteccion \n cubrebocas,cuidado \n cubrebocas,salud \n cubrebocas,limpieza \n cubrebocas,germenes \n cubrebocas,nr \n cubrebocas,nr \n cubrebocas,nr \n cubrebocas,nr \n cubrebocas,...,saludable \n ejercicio,estimulacion \n ejercicio,neuronal \n ejercicio,conexion \n ejercicio,longevidad \n ejercicio,fortaleza \n ejercicio,nr \n ejercicio,nr \n ejercicio,nr \n ejercicio,nr \n ejercicio
2,69,saludable \n ejercicio,salud \n ejercicio,bueno \n ejercicio,forma \n ejercicio,deporte \n ejercicio,aparatos \n ejercicio,gimnasio \n ejercicio,correr \n ejercicio,nr \n ejercicio,...,doctor \n cubrebocas,hospital \n cubrebocas,enfermedad \n cubrebocas,covid \n cubrebocas,pandemia \n cubrebocas,necesario \n cubrebocas,util \n cubrebocas,incomodo \n cubrebocas,nr \n cubrebocas,nr \n cubrebocas
3,70,enfermedad \n obesidad,alimentacion \n obesidad,comida \n obesidad,consumo \n obesidad,atracon \n obesidad,descompensacion \n obesidad,ejercicio \n obesidad,activacion \n obesidad,deporte \n obesidad,...,enfermedad \n covid,virus \n covid,pandemia \n covid,situacion \n covid,global \n covid,estres \n covid,ansiedad \n covid,muertes \n covid,nr \n covid,nr \n covid
4,71,ballet \n ejercicio,figura \n ejercicio,pilates \n ejercicio,peso \n ejercicio,actividad \n ejercicio,salud \n ejercicio,correr \n ejercicio,aerobico \n ejercicio,nr \n ejercicio,...,covid \n cubrebocas,pandemia \n cubrebocas,prevencion \n cubrebocas,enfermedad \n cubrebocas,cuarentena \n cubrebocas,necesario \n cubrebocas,salud \n cubrebocas,incomodo \n cubrebocas,nr \n cubrebocas,nr \n cubrebocas
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
716,849,bienestar \n ejercicio,salud \n ejercicio,terapia \n ejercicio,motivacion \n ejercicio,dedicacion \n ejercicio,esfuerzo \n ejercicio,tiempo \n ejercicio,nr \n ejercicio,nr \n ejercicio,...,pandemia \n covid,encierro \n covid,enfermedad \n covid,trauma \n covid,miedo \n covid,riesgo \n covid,crecimiento \n covid,conflicto \n covid,insertidumbre \n covid,nr \n covid
718,851,tratamiento \n vacuna,medicamento \n vacuna,dolor \n vacuna,doctor \n vacuna,nr \n vacuna,nr \n vacuna,nr \n vacuna,nr \n vacuna,nr \n vacuna,...,enfermedades \n obesidad,mala alimentacion \n obesidad,nr \n obesidad,nr \n obesidad,nr \n obesidad,nr \n obesidad,nr \n obesidad,nr \n obesidad,nr \n obesidad,nr \n obesidad
719,852,burla \n obesidad,tristeza \n obesidad,cansancio \n obesidad,complicado \n obesidad,soledad \n obesidad,nr \n obesidad,nr \n obesidad,nr \n obesidad,nr \n obesidad,...,conocimiento \n vacuna,ciencia \n vacuna,termino \n vacuna,autocuidado \n vacuna,nr \n vacuna,nr \n vacuna,nr \n vacuna,nr \n vacuna,nr \n vacuna,nr \n vacuna
720,853,higiene \n cubrebocas,covid \n cubrebocas,salud \n cubrebocas,barrera \n cubrebocas,estornudo \n cubrebocas,distanciamiento \n cubrebocas,azul \n cubrebocas,nr \n cubrebocas,nr \n cubrebocas,...,necesario \n ejercicio,disciplina \n ejercicio,salud \n ejercicio,bueno \n ejercicio,consciencia \n ejercicio,riguroso \n ejercicio,tiempo \n ejercicio,nr \n ejercicio,nr \n ejercicio,nr \n ejercicio


In [187]:
largo = datos_rondas.melt(id_vars='id',
                          var_name='campo',
                          value_name='valor')
largo

,id,campo,valor
0,67,w4576[SQ001],salud \n ejercicio
1,68,w4576[SQ001],proteccion \n cubrebocas
2,69,w4576[SQ001],saludable \n ejercicio
3,70,w4576[SQ001],enfermedad \n obesidad
4,71,w4576[SQ001],ballet \n ejercicio
...,...,...,...
37375,849,y65[SQ010],nr \n covid
37376,851,y65[SQ010],nr \n obesidad
37377,852,y65[SQ010],nr \n vacuna
37378,853,y65[SQ010],nr \n ejercicio


In [188]:
largo['posicion'] = largo['campo'].str.extract(r'\[.*(\d{3})\]').astype(int)
largo.drop(columns=['campo'], inplace=True)
largo

,id,valor,posicion
0,67,salud \n ejercicio,1
1,68,proteccion \n cubrebocas,1
2,69,saludable \n ejercicio,1
3,70,enfermedad \n obesidad,1
4,71,ballet \n ejercicio,1
...,...,...,...
37375,849,nr \n covid,10
37376,851,nr \n obesidad,10
37377,852,nr \n vacuna,10
37378,853,nr \n ejercicio,10


In [189]:
largo['estimulo'] = largo['valor'].str.extract(r'.*\s\n\s(\w+)')
largo['valor'] = largo['valor'].str.extract(r'(.*)\s\n\s\w+')
largo.rename(columns={'valor': 'respuesta'}, inplace=True)
assert largo['estimulo'].nunique() == 6
largo

,id,respuesta,posicion,estimulo
0,67,salud,1,ejercicio
1,68,proteccion,1,cubrebocas
2,69,saludable,1,ejercicio
3,70,enfermedad,1,obesidad
4,71,ballet,1,ejercicio
...,...,...,...,...
37375,849,nr,10,covid
37376,851,nr,10,obesidad
37377,852,nr,10,vacuna
37378,853,nr,10,ejercicio


In [190]:
largo['estimulo'] += largo['posicion'].astype(str).str.zfill(2)
largo = (largo
         .filter(['id', 'estimulo', 'respuesta'], axis=1)
         .sort_values(by=['id', 'estimulo'], ignore_index=True)
         )
largo

,id,estimulo,respuesta
0,67,covid01,cuarentena
1,67,covid02,ansiedad
2,67,covid03,preocupacion
3,67,covid04,miedo
4,67,covid05,imparable
...,...,...,...
37375,855,vacuna06,dolor
37376,855,vacuna07,miedo
37377,855,vacuna08,aguja
37378,855,vacuna09,liquido


Datos de obesidad duplicados para algunos id

In [191]:
#largo.groupby(['id', 'estimulo', 'respuesta']).size().drop_duplicates()
#largo[largo['id'].between(797, 799)].pivot(index='id', columns='estimulo', values='respuesta')
ancho = largo[~largo['id'].isin([796, 534])].pivot(index='id', columns='estimulo', values='respuesta')

In [192]:
ancho

estimulo,covid01,covid02,covid03,covid04,covid05,covid06,covid07,covid08,covid09,covid10,...,vacuna01,vacuna02,vacuna03,vacuna04,vacuna05,vacuna06,vacuna07,vacuna08,vacuna09,vacuna10
id,,,,,,,,,,,,,,,,,,,,,
67,cuarentena,ansiedad,preocupacion,miedo,imparable,encierro,mundial,cuidarse,salud,nr,...,necesaria,social,pelea,lucha,guerra,preocupante,experimento,ayuda,nr,nr
68,muerte,falta de proteccion,no hay libertad,enfermedad,sana distancia,ansiedad,cubrebocas,guantes,nr,nr,...,salud,proteccion,anticuerpos,oportunidad,longevidad,saludable,nr,nr,nr,nr
69,virus,enfermedad,muertes,pandemia,cuarentena,sisntomas,doctores,respiracion,china,nr,...,cura,tratamiento,medicina,enfermedad,virus,ahuja,util,covid,dolor,nr
70,enfermedad,virus,pandemia,situacion,global,estres,ansiedad,muertes,nr,nr,...,responsabilidad,cura,virus,salud,enfermedad,sociedad,pandemia,tiempo,nr,nr
71,enfermedad,pandemia,cuarentena,mundo,doctores,vacuna,china,nr,nr,nr,...,enfermedad,pandemia,covid,salud,cuarentena,inyeccion,doctores,hospital,personas,nr
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
849,pandemia,encierro,enfermedad,trauma,miedo,riesgo,crecimiento,conflicto,insertidumbre,nr,...,proteccion,salud,defensa,barrera,problema,miedo,insertidumbre,negacion,nr,nr
851,preocupacion,contagios,estres,muertes,confinamiento,enfermedad,nr,nr,nr,nr,...,tratamiento,medicamento,dolor,doctor,nr,nr,nr,nr,nr,nr
852,muerte,depresion,clausurado,casa,cubrebocas,aire,nr,nr,nr,nr,...,conocimiento,ciencia,termino,autocuidado,nr,nr,nr,nr,nr,nr


In [193]:
datos = (datos
         .drop(columns=columnas_rondas)
         .merge(ancho, on='id', how='left')
         )
datos

,id,submitdate,lastpage,startlanguage,seed,startdate,datestamp,encuestador,conset,q35,...,vacuna01,vacuna02,vacuna03,vacuna04,vacuna05,vacuna06,vacuna07,vacuna08,vacuna09,vacuna10
0,67,2021-03-27 13:32:28,5,es-MX,909191565,2021-03-27 13:03:10,2021-03-27 13:32:28,A13,Y,Y,...,necesaria,social,pelea,lucha,guerra,preocupante,experimento,ayuda,nr,nr
1,68,2021-03-27 19:38:43,5,es-MX,99489830,2021-03-27 19:07:21,2021-03-27 19:38:43,A13,Y,Y,...,salud,proteccion,anticuerpos,oportunidad,longevidad,saludable,nr,nr,nr,nr
2,69,2021-03-29 11:47:46,5,es-MX,1592450376,2021-03-29 11:04:42,2021-03-29 11:47:46,A07,Y,Y,...,cura,tratamiento,medicina,enfermedad,virus,ahuja,util,covid,dolor,nr
3,70,2021-03-29 12:37:20,5,es-MX,762883078,2021-03-29 12:04:01,2021-03-29 12:37:20,A04,Y,Y,...,responsabilidad,cura,virus,salud,enfermedad,sociedad,pandemia,tiempo,nr,nr
4,71,2021-03-29 14:33:34,5,es-MX,1746037044,2021-03-29 14:00:48,2021-03-29 14:33:34,A03,Y,Y,...,enfermedad,pandemia,covid,salud,cuarentena,inyeccion,doctores,hospital,personas,nr
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
717,851,2022-10-29 17:28:54,5,es-MX,584177600,2022-10-29 17:07:05,2022-10-29 17:28:54,A05,Y,Y,...,tratamiento,medicamento,dolor,doctor,nr,nr,nr,nr,nr,nr
718,852,2022-10-29 18:20:26,5,es-MX,921555126,2022-10-29 18:02:41,2022-10-29 18:20:26,A05,Y,Y,...,conocimiento,ciencia,termino,autocuidado,nr,nr,nr,nr,nr,nr
719,853,2022-10-29 20:23:28,5,es-MX,114360708,2022-10-29 20:02:20,2022-10-29 20:23:28,A05,Y,Y,...,proteccion,obligatorio,prevencion,util,necesario,dolor,nr,nr,nr,nr
720,854,2022-10-31 19:02:54,5,es-MX,2117179292,2022-10-31 18:32:54,2022-10-31 19:02:54,A11,Y,Y,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [194]:
mapear_codigos = functools.partial(auxiliar_4.mapear_codigos, datos, mapeo_respuestas)
llenar_respuesta_libre = functools.partial(auxiliar_4.llenar_respuesta_libre, datos)
llenar_vacios = functools.partial(auxiliar_4.llenar_vacios, datos)
mostrar_unicos = functools.partial(auxiliar_4.mostrar_unicos, datos)
contar_faltantes = functools.partial(auxiliar_4.contar_faltantes, datos)
unificar_observaciones = functools.partial(auxiliar_4.unificar_observaciones, datos)

# Trabajos

Los trabajos tienen dos niveles de organización:
- Nivel 1: categoría (tipo de trabajo)
- Nivel 2: ocupación (el trabajo en sí mismo)

## areatrabajo (nivel 1)

Las posibles respuestas de `areatrabajo` corresponden al nivel 1.

In [195]:
mapeo_respuestas['areatrabajo']

{'A02': 'Actividades deportivas_1',
 'A03': 'Artista_2',
 'A04': 'Chofer_3',
 'A05': 'Comerciante sector formal_4',
 'A06': 'Comerciante sector informal_5',
 'A07': 'Empresario_6',
 'A08': 'Comercio Virtual_7',
 'A09': 'Atención a clientes_8',
 'A10': 'Administrativo_9',
 'A11': 'Gubernamental administrativo_10',
 'A12': 'Empleado gubernamental_11',
 'A13': 'Operador de maquinaria móvil_12',
 'A14': 'Personal técnico_13',
 'A15': 'Área educativa_14',
 'A16': 'Profesionista independiente_15',
 'A17': 'Seguridad_16',
 'A18': 'Área de la salud_17',
 'A19': 'Construcción_18',
 'A20': 'Fábrica_19'}

Los valores registrados para el área de trabajo son:

In [196]:
mostrar_unicos('areatrabajo')

0     -oth-
1       A02
2       A03
3       A04
4       A05
5       A06
6       A07
7       A08
8       A09
9       A10
10      A12
11      A13
12      A14
13      A15
14      A16
15      A18
16      A20
17      NaN
Name: areatrabajo, dtype: object

Entonces las respuestas únicas para el área de trabajo son:

In [197]:
llenar_respuesta_libre('areatrabajo')
llenar_vacios('areatrabajo', 'ninguna')
mapear_codigos('areatrabajo')
datos['areatrabajo'] = datos['areatrabajo'].str.strip().str.lower()
mostrar_unicos('areatrabajo')

/Users/erickavendanogarcia/Downloads/servicio_social/encuestas/auxiliar_4.py:143: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[columna].fillna(valor, inplace=True)


0                 actividades deportivas_1
1     administradora de servicios de salud
2                         administrativo_9
3         area administrativa de panadería
4                                artista_2
5                                     aseo
6                    atención a clientes_8
7                      atiende tienda neto
8                   ayudante de psicología
9                       ayudante de sastre
10          ayudante empresa de publicidad
11         ayudante general tienda de ropa
12                                 becaria
13                               cafeteria
14                       cafeteria escolar
15             caseta de cobro de un bazar
16                                chofer_3
17                                  cocina
18             comerciante sector formal_4
19           comerciante sector informal_5
20                      comercio virtual_7
21                                empleado
22               empleado gubernamental_11
23         

## Ocupaciones (nivel 2)

Si una persona indica su área de trabajo dentro de la categoría de nivel 1, entonces se le pregunta por su ocupación (nivel 2) dentro de esa categoría.
Para cada categoría se lidia con el nivel 2 a través de dos columnas:
- Ocupación: se elige entre las opciones predeterminadas. A estas columnas se les denominará **base**.
- Si la ocupación de la persona no se encuentra dentro de las opciones predeterminadas, entonces elige "otra" y registra su ocupación en una columna con el sufijo `"[other]"`. A estas columnas se les denominará **sufijo**

Por tanto, cada categoría de nivel 1 tiene asociadas dos columnas de ocupación (nivel 2).

### Columnas base

Primero se identifica las columnas base. Casi todas ellas tienen el sufijo `q0r` seguido de un número. Sin embargo, hay algunas excepciones: `deport`, `construcion`, `fabrica` y `formal`

In [198]:
columnas_base = diccionario.index[diccionario.index.str.contains('[a-zA-Z]+q0r\d+$')].tolist()
columnas_base += ['deport', 'construccion', 'fabricaq57', 'formal21']
columnas_base.sort()
columnas_base

<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
/var/folders/hb/c3zh8bns0mx4nw98_6k7rx8m0000gn/T/ipykernel_4464/1050578966.py:1: SyntaxWarning: invalid escape sequence '\d'
  columnas_base = diccionario.index[diccionario.index.str.contains('[a-zA-Z]+q0r\d+$')].tolist()


['agriculturaq0r411',
 'artistaq0r621',
 'atenciclientesq0r13',
 'choferq0r727',
 'construccion',
 'deport',
 'dministratq0r139',
 'educacionq0r857',
 'empleadogubq0r22',
 'empresaq0r18',
 'fabricaq57',
 'formal21',
 'gubadmiq0r335',
 'indepq0r708',
 'informalq0r332',
 'internetq0r391',
 'operadormaquiq0r947',
 'persontecnq0r477',
 'saludq0r580',
 'segurq0r642']

In [199]:
datos[columnas_base]

,agriculturaq0r411,artistaq0r621,atenciclientesq0r13,choferq0r727,construccion,deport,dministratq0r139,educacionq0r857,empleadogubq0r22,empresaq0r18,fabricaq57,formal21,gubadmiq0r335,indepq0r708,informalq0r332,internetq0r391,operadormaquiq0r947,persontecnq0r477,saludq0r580,segurq0r642
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-oth-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
717,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
718,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
719,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
720,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [200]:
for col_carrera in columnas_base:
    mapear_codigos(col_carrera)

/Users/erickavendanogarcia/Downloads/servicio_social/encuestas/auxiliar_4.py:99: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[columna].replace(mapeo[columna], inplace=True)


In [201]:
datos[columnas_base]

,agriculturaq0r411,artistaq0r621,atenciclientesq0r13,choferq0r727,construccion,deport,dministratq0r139,educacionq0r857,empleadogubq0r22,empresaq0r18,fabricaq57,formal21,gubadmiq0r335,indepq0r708,informalq0r332,internetq0r391,operadormaquiq0r947,persontecnq0r477,saludq0r580,segurq0r642
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-oth-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
717,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
718,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
719,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
720,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Columnas sufijo
El objetivo es pasar los datos de las columnas con sufijo a las columnas base.
No todas las columnas tienen el mismo sufijo, pero las dos opciones son `"[other]"` y `"[comment]"`:

In [202]:
columnas_sufijo = []
for col in columnas_base:
    if col + '[other]' in datos.columns:
        columnas_sufijo.append(col + '[other]')
    elif col + '[comment]' in datos.columns:
        columnas_sufijo.append(col + '[comment]')
assert len(columnas_sufijo) == len(columnas_base)
columnas_sufijo.sort()
columnas_sufijo

['agriculturaq0r411[other]',
 'artistaq0r621[other]',
 'atenciclientesq0r13[other]',
 'choferq0r727[other]',
 'construccion[other]',
 'deport[other]',
 'dministratq0r139[other]',
 'educacionq0r857[other]',
 'empleadogubq0r22[comment]',
 'empresaq0r18[other]',
 'fabricaq57[other]',
 'formal21[other]',
 'gubadmiq0r335[other]',
 'indepq0r708[other]',
 'informalq0r332[other]',
 'internetq0r391[other]',
 'operadormaquiq0r947[comment]',
 'persontecnq0r477[comment]',
 'saludq0r580[other]',
 'segurq0r642[other]']

Ahora la limpieza es directa:

In [203]:
for col_con_sufijo in columnas_sufijo:
    columna_sin_sufijo = re.sub(r'\[\w+\]', '', col_con_sufijo)
    datos[columna_sin_sufijo] = datos[columna_sin_sufijo].combine_first(datos[col_con_sufijo])
    datos[columna_sin_sufijo] = np.where(datos[columna_sin_sufijo] == '-oth-',
                                         datos[col_con_sufijo],
                                         datos[columna_sin_sufijo])
    datos.drop(columns=col_con_sufijo, inplace=True)

### Resultado

In [204]:
datos[columnas_base]

,agriculturaq0r411,artistaq0r621,atenciclientesq0r13,choferq0r727,construccion,deport,dministratq0r139,educacionq0r857,empleadogubq0r22,empresaq0r18,fabricaq57,formal21,gubadmiq0r335,indepq0r708,informalq0r332,internetq0r391,operadormaquiq0r947,persontecnq0r477,saludq0r580,segurq0r642
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CLASES DE FRANCES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
717,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
718,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
719,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
720,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Asignación trabajo

Ahora que toda la información sobre las distintas ocupaciones está en las columnas base, se puede crear una columna que combine la información de todas ellas, la cual se llamará `'trabajo'`.

In [205]:
datos['trabajo'] = pd.Series(dtype='string', index=datos.index)  # columna nueva
for columna in columnas_base:
    datos['trabajo'] = datos['trabajo'].combine_first(datos[columna])
    datos.drop(columns=columna, inplace=True)
datos['trabajo'] = datos['trabajo'].combine_first(datos['areatrabajo'])
datos.drop(columns='areatrabajo', inplace=True)
datos.drop(columns='vendinformalq0r332', inplace=True)
datos.drop(columns='vendinformalq0r332[other]', inplace=True)
datos['trabajo'] = datos['trabajo'].str.strip().str.lower()
mostrar_unicos('trabajo')

/var/folders/hb/c3zh8bns0mx4nw98_6k7rx8m0000gn/T/ipykernel_4464/3838770192.py:3: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  datos['trabajo'] = datos['trabajo'].combine_first(datos[columna])


0                            a1
1                     abarrotes
2                     abogado_1
3      actividades comunitarias
4                         actor
                 ...           
114            vendedor general
115               venta de ropa
116          venta por catalogo
117                        yoga
118           área educativa_14
Name: trabajo, Length: 119, dtype: object

In [206]:
contar_faltantes('trabajo')

0

Dentro de los trabajos únicos solo es necesario lidiar con aquellos que no están considerados en el diccionario de respuestas, los cuales tienen un sufijo compuesto por un guión bajo y un número

In [207]:
trabajos_nuevos = datos['trabajo'][~datos['trabajo'].str.contains('_\d+$|ninguna')].reset_index(drop=True)
trabajos_nuevos

<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
/var/folders/hb/c3zh8bns0mx4nw98_6k7rx8m0000gn/T/ipykernel_4464/760739842.py:1: SyntaxWarning: invalid escape sequence '\d'
  trabajos_nuevos = datos['trabajo'][~datos['trabajo'].str.contains('_\d+$|ninguna')].reset_index(drop=True)


0                                     clases de frances
1      vendedor de productos por internet (mercado pago
2                                                  ropa
3      vendedor de productos por internet (mercado pago
4                        ayudante empresa de publicidad
                             ...                       
117                                         laboratorio
118                                          entrenador
119                                        optometrista
120                                             iglesia
121                                personal de limpieza
Name: trabajo, Length: 122, dtype: object

## Corrección de trabajos

In [208]:
import json

with open('codigos_preguntas/propuesta.json', 'r') as f:
    propuesta = json.load(f)
propuesta

{'actividades deportivas_1': ['entrenador',
  'instructor',
  'profesor de clases deportivas'],
 'artista_2': ['actor',
  'ballet',
  'cantante',
  'diseñador',
  'fotógrafo',
  'modelo',
  'músico',
  'pintor'],
 'chofer_3': ['camiones', 'microbús', 'taxista', 'uber'],
 'comerciante sector informal_4': ['bazar',
  'flores y productos en un puesto no establecido',
  'ropa',
  'vendedor',
  'vendedor por cuenta propia (negocio propio)',
  'venta de cigarros'],
 'comerciante sector formal_5': ['periódicos flores y productos en un puesto no establecido',
  'productos en un puesto no establecido',
  'taquero',
  'vendedor de comida'],
 'empresario_6': ['asesor',
  'consultor',
  'director',
  'dueño',
  'fundación',
  'gerente'],
 'comercio virtual_7': ['vendedor de productos por internet (mercado pago, amazon, instagram, etc.)'],
 'atención a clientes_8': ['atención a clientes',
  'cajero',
  'call center',
  'promotor de seguros',
  'recepcionista',
  'telefonía',
  'vendedor de tiendas 

In [209]:
trabajos_oficiales = pd.Series(dtype='string')
for t in propuesta.values():
    trabajos_oficiales = pd.concat([trabajos_oficiales, pd.Series(t)])
trabajos_oficiales.reset_index(drop=True, inplace=True)
trabajos_oficiales

0                         entrenador
1                         instructor
2      profesor de clases deportivas
3                              actor
4                             ballet
                   ...              
119                           sastre
120                         zapatero
121                         zapatero
122                         mecánico
123                          ninguna
Length: 124, dtype: object

In [210]:
#group_similar_strings(trabajos_nuevos, min_similarity=0.5, ignore_index=True)

In [211]:
correccion = match_most_similar(trabajos_oficiales,
                                trabajos_nuevos,
                                min_similarity=0.1,
                                ignore_index=True)
comparar = pd.DataFrame({'original': trabajos_nuevos, 'correccion': correccion})
comparar['hubo_correccion'] = comparar['original'].ne(comparar['correccion'])
comparar.sort_values(by='hubo_correccion', ascending=False)

/Users/erickavendanogarcia/Downloads/servicio_social/venv/lib/python3.12/site-packages/string_grouper/string_grouper.py:694: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  elif series_to_test.to_frame().applymap(


,original,correccion,hubo_correccion
0,clases de frances,dar clases,True
49,cafeteria,cafetería,True
84,soporte técnico,técnico en educación (educador o guardería),True
82,taqueria,taquero,True
81,vendedor de productos por internet (mercado pago,vendedor de productos por internet (mercado pa...,True
...,...,...,...
63,finanzas,finanzas,False
19,idiomas,idiomas,False
94,a1,a1,False
18,marketing,marketing,False


In [212]:
mapeo_correcciones = dict(zip(trabajos_nuevos, correccion))
mapeo_correcciones

{'clases de frances': 'dar clases',
 'vendedor de productos por internet (mercado pago': 'vendedor de productos por internet (mercado pago, amazon, instagram, etc.)',
 'ropa': 'ropa',
 'ayudante empresa de publicidad': 'publicista',
 'cajera': 'cajero',
 'caseta de cobro de un bazar': 'bazar',
 'farmacia': 'farmacia',
 'becaria': 'becario',
 'a1': 'a1',
 'cigarros electronicos': 'venta de cigarros',
 'univercidad': 'universidad',
 'reclutamientos': 'reclutador',
 'psicologia': 'psicólogo',
 'fundacion proyecto duq': 'fundación',
 'marketing': 'marketing',
 'idiomas': 'idiomas',
 'cafeteria': 'cafetería',
 'trabaja en una farmacia': 'farmacia',
 'modelo': 'modelo',
 'regularizacion': 'regularización',
 'escuela': 'escuela',
 'actor': 'actor',
 'artesanías': 'artesano',
 'empleado': 'empleado',
 'tienda. atención a clientes': 'atención a clientes',
 'gerente': 'gerente',
 'actividades comunitarias': 'actividades comunitarias',
 'nutriologa': 'nutriólogo',
 'consultor en riesgos': 'consul

In [213]:
datos['trabajo'] = datos['trabajo'].replace(mapeo_correcciones)
datos['trabajo'] = datos['trabajo'].str.replace(r'_\d+', '', regex=True)
mostrar_unicos('trabajo')

0                                                    a1
1                                             abarrotes
2                                               abogado
3                              actividades comunitarias
4                                                 actor
                            ...                        
76    vendedor de productos por internet (mercado pa...
77                  vendedor de tiendas departamentales
78          vendedor por cuenta propia (negocio propio)
79                                    venta de cigarros
80                                       área educativa
Name: trabajo, Length: 81, dtype: object

In [214]:
categorias = pd.Series(dtype='string', index=datos.index)
for i, ocupacion in datos['trabajo'][datos['trabajo'].notna()].items():
    registrada = False
    if ocupacion == 'ninguna':
        categorias.loc[i] = 'desconocido_99'
        regisrada = True
    else:
        for categoria, ocupaciones_dentro_de_categoria in propuesta.items():
            if ocupacion in ocupaciones_dentro_de_categoria:
                categorias.loc[i] = categoria
                registrada = True
                break
        if not registrada:
            print(f'La ocupacion {ocupacion} es desconocida')
            categorias.loc[i] = 'desconocido_99'


La ocupacion a1 es desconocida
La ocupacion a1 es desconocida
La ocupacion escuela es desconocida
La ocupacion a1 es desconocida
La ocupacion a1 es desconocida
La ocupacion a1 es desconocida
La ocupacion entrenador de gimnasio es desconocida
La ocupacion pizzeria es desconocida
La ocupacion a1 es desconocida
La ocupacion a1 es desconocida
La ocupacion finanzas es desconocida
La ocupacion a1 es desconocida
La ocupacion a1 es desconocida
La ocupacion a1 es desconocida
La ocupacion auxiliar es desconocida
La ocupacion a1 es desconocida
La ocupacion a1 es desconocida
La ocupacion a1 es desconocida
La ocupacion auxiliar es desconocida
La ocupacion a1 es desconocida
La ocupacion a1 es desconocida
La ocupacion playeras es desconocida
La ocupacion iglesia es desconocida


In [215]:
categorias.drop_duplicates().sort_values().reset_index(drop=True)

0                  actividades deportivas_1
1     actividades elementales y de apoyo_20
2                          administrativo_9
3                                 artista_2
4                     atención a clientes_8
5                                  chofer_3
6               comerciante sector formal_5
7             comerciante sector informal_4
8                        comercio virtual_7
9                            desconocido_99
10                empleado gubernamental_11
11                             empresario_6
12                               fábrica_18
13          gubernamental administrativo_10
14                               oficios_21
15           profesionista independiente_14
16                      área de la salud_16
17                        área educativa_13
dtype: string

In [216]:
datos['categoria'] = categorias

In [217]:
datos[['trabajo', 'categoria']]

,trabajo,categoria
0,ninguna,desconocido_99
1,ninguna,desconocido_99
2,ninguna,desconocido_99
3,ninguna,desconocido_99
4,dar clases,área educativa_13
...,...,...
717,ninguna,desconocido_99
718,ninguna,desconocido_99
719,ninguna,desconocido_99
720,ninguna,desconocido_99


In [218]:
mapeo_codigos_categoria = {}
for cat in categorias.drop_duplicates():
    numero = re.findall(r'_(\d+)', cat)[0]
    mapeo_codigos_categoria[cat] = 'A' + str(numero).zfill(2)
mapeo_codigos_categoria

{'desconocido_99': 'A99',
 'área educativa_13': 'A13',
 'comercio virtual_7': 'A07',
 'comerciante sector formal_5': 'A05',
 'comerciante sector informal_4': 'A04',
 'profesionista independiente_14': 'A14',
 'atención a clientes_8': 'A08',
 'actividades elementales y de apoyo_20': 'A20',
 'administrativo_9': 'A09',
 'empresario_6': 'A06',
 'empleado gubernamental_11': 'A11',
 'área de la salud_16': 'A16',
 'artista_2': 'A02',
 'oficios_21': 'A21',
 'fábrica_18': 'A18',
 'actividades deportivas_1': 'A01',
 'gubernamental administrativo_10': 'A10',
 'chofer_3': 'A03'}

In [219]:
datos['categoria'].replace(mapeo_codigos_categoria, inplace=True)
datos['categoria'].drop_duplicates().sort_values().reset_index(drop=True)

/var/folders/hb/c3zh8bns0mx4nw98_6k7rx8m0000gn/T/ipykernel_4464/457275959.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  datos['categoria'].replace(mapeo_codigos_categoria, inplace=True)


0     A01
1     A02
2     A03
3     A04
4     A05
5     A06
6     A07
7     A08
8     A09
9     A10
10    A11
11    A13
12    A14
13    A16
14    A18
15    A20
16    A21
17    A99
Name: categoria, dtype: string

In [220]:
mapeo_codigos_ocupaciones = {}
for cat in categorias.drop_duplicates():
    codigo_cat = mapeo_codigos_categoria[cat]
    ocupaciones_en_categoria = datos['trabajo'][
        datos['categoria'] == codigo_cat].drop_duplicates().sort_values().reset_index(drop=True)
    for i, ocupacion in ocupaciones_en_categoria.items():
        mapeo_codigos_ocupaciones[ocupacion] = codigo_cat + str(i + 1).zfill(2)
mapeo_codigos_ocupaciones

{'a1': 'A9901',
 'auxiliar': 'A9902',
 'entrenador de gimnasio': 'A9903',
 'escuela': 'A9904',
 'finanzas': 'A9905',
 'iglesia': 'A9906',
 'ninguna': 'A9907',
 'pizzeria': 'A9908',
 'playeras': 'A9909',
 'dar clases': 'A1301',
 'docente': 'A1302',
 'educación básica': 'A1303',
 'estudiante': 'A1304',
 'idiomas': 'A1305',
 'investigador-profesor en universidad/preparatoria': 'A1306',
 'profesor': 'A1307',
 'regularización': 'A1308',
 'secundaria': 'A1309',
 'técnico en educación (educador o guardería)': 'A1310',
 'universidad': 'A1311',
 'área educativa': 'A1312',
 'vendedor de productos por internet (mercado pago, amazon, instagram, etc.)': 'A0701',
 'taquero': 'A0501',
 'vendedor de comida': 'A0502',
 'bazar': 'A0401',
 'flores y productos en un puesto no establecido': 'A0402',
 'ropa': 'A0403',
 'vendedor por cuenta propia (negocio propio)': 'A0404',
 'venta de cigarros': 'A0405',
 'abogado': 'A1401',
 'contador': 'A1402',
 'marketing': 'A1403',
 'psicólogo': 'A1404',
 'publicista': 

In [221]:
datos[['trabajo', 'categoria']]

,trabajo,categoria
0,ninguna,A99
1,ninguna,A99
2,ninguna,A99
3,ninguna,A99
4,dar clases,A13
...,...,...
717,ninguna,A99
718,ninguna,A99
719,ninguna,A99
720,ninguna,A99


In [233]:
datos.iloc[63, datos.columns.get_loc('trabajo')] = 'vendedor por cuenta propia (negocio propio)'
datos.iloc[296, datos.columns.get_loc('trabajo')] = 'vendedor por cuenta propia (negocio propio)'
datos.iloc[114, datos.columns.get_loc('trabajo')] = 'vendedor por cuenta propia (negocio propio)'
datos.iloc[290, datos.columns.get_loc('trabajo')] = 'vendedor por cuenta propia (negocio propio)'
datos.iloc[294, datos.columns.get_loc('trabajo')] = 'vendedor por cuenta propia (negocio propio)'
datos.iloc[296, datos.columns.get_loc('trabajo')] = 'vendedor por cuenta propia (negocio propio)'
datos.iloc[390, datos.columns.get_loc('trabajo')] = 'legisladora'
datos.iloc[428, datos.columns.get_loc('trabajo')] = 'vendedor por cuenta propia (negocio propio)'
datos.iloc[483, datos.columns.get_loc('trabajo')] = 'vendedor por cuenta propia (negocio propio)'
datos.iloc[485, datos.columns.get_loc('trabajo')] = 'vendedor por cuenta propia (negocio propio)'
datos.iloc[564, datos.columns.get_loc('trabajo')] = 'técnico'
datos.iloc[568, datos.columns.get_loc('trabajo')] = 'administrativo'
datos.iloc[621, datos.columns.get_loc('trabajo')] = 'vendedor por cuenta propia (negocio propio)'
datos.iloc[651, datos.columns.get_loc('trabajo')] = 'administrativo'
datos.iloc[665, datos.columns.get_loc('trabajo')] = 'vendedor por cuenta propia (negocio propio)'
datos.iloc[670, datos.columns.get_loc('trabajo')] = 'vendedor por cuenta propia (negocio propio)'



In [234]:
datos.iloc[63, datos.columns.get_loc('categoria')] = 'A04'
datos.iloc[296, datos.columns.get_loc('categoria')] = 'A04'
datos.iloc[114, datos.columns.get_loc('categoria')] = 'A04'
datos.iloc[290, datos.columns.get_loc('categoria')] = 'A04'
datos.iloc[294, datos.columns.get_loc('categoria')] = 'A04'
datos.iloc[296, datos.columns.get_loc('categoria')] = 'A04'
datos.iloc[390, datos.columns.get_loc('categoria')] = 'A11'
datos.iloc[428, datos.columns.get_loc('categoria')] = 'A04'
datos.iloc[483, datos.columns.get_loc('categoria')] = 'A04'
datos.iloc[485, datos.columns.get_loc('categoria')] = 'A04'
datos.iloc[564, datos.columns.get_loc('categoria')] = 'A12'
datos.iloc[568, datos.columns.get_loc('categoria')] = 'A06'
datos.iloc[621, datos.columns.get_loc('categoria')] = 'A04'
datos.iloc[651, datos.columns.get_loc('categoria')] = 'A06'
datos.iloc[665, datos.columns.get_loc('categoria')] = 'A04'
datos.iloc[670, datos.columns.get_loc('categoria')] = 'A04'


In [235]:
archivo_json = 'Trabajos_familia.json'

# Leer el archivo JSON y convertirlo en un diccionario
with open(archivo_json, 'r', encoding='utf-8') as f:
    mapeo_reorganizado_importado = json.load(f)
mapeo_reorganizado_importado

{'actividad religiosa': 'A9901',
 'belleza': 'A9902',
 'cacique de rancho': 'A9903',
 'capataz': 'A9904',
 'carbón': 'A9905',
 'club': 'A9906',
 'coaching': 'A9907',
 'desconoce': 'A9908',
 'desconocido': 'A9908',
 'se desconoce': 'A9908',
 'distribuidor': 'A9909',
 'editorial': 'A9910',
 'em': 'A9911',
 'estacion de radio': 'A9912',
 'evaciones psicologicas': 'A9913',
 'facturas': 'A9914',
 'guantes': 'A9915',
 'hace trofeos y medallas por mayoreo': 'A9916',
 'homeopatia': 'A9917',
 'jubilado': 'A9918',
 'lo que hacía': 'A9919',
 'menos infractores e inadaptados': 'A9920',
 'ninguna': 'A9921',
 'no aplica': 'A9922',
 'no hay información': 'A9923',
 'no lo conoció': 'A9924',
 'no recuerdo': 'A9925',
 'no sabe': 'A9925',
 'no se': 'A9925',
 'no se sabe': 'A9925',
 'no sé': 'A9925',
 'omitir': 'A9926',
 'orador': 'A9927',
 'pastor (religion)': 'A9928',
 'pensionada': 'A9929',
 'pensionado': 'A9929',
 'pensionado de pgr (mecanico de helicopteros)': 'A9929',
 'por contrato': 'A9930',
 'pre

# Era hasta aquí

In [236]:
datos['trabajo'].replace(mapeo_reorganizado_importado, inplace=True)
datos['trabajo'].drop_duplicates().sort_values().reset_index(drop=True)

/var/folders/hb/c3zh8bns0mx4nw98_6k7rx8m0000gn/T/ipykernel_4464/2721062508.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  datos['trabajo'].replace(mapeo_reorganizado_importado, inplace=True)


0     A0103
1     A0104
2     A0201
3     A0207
4     A0208
      ...  
70    A2109
71    A2121
72    A2129
73    A9921
74    A9944
Name: trabajo, Length: 75, dtype: object

In [237]:
datos[['trabajo', 'categoria']]

,trabajo,categoria
0,A9921,A99
1,A9921,A99
2,A9921,A99
3,A9921,A99
4,A1303,A13
...,...,...
717,A9921,A99
718,A9921,A99
719,A9921,A99
720,A9921,A99


In [238]:
datos.iloc[659, datos.columns.get_loc('trabajo')] = 'A9944'
datos.iloc[566, datos.columns.get_loc('trabajo')] = 'A9944'
datos.iloc[327, datos.columns.get_loc('trabajo')] = 'A0103'
datos.iloc[170, datos.columns.get_loc('trabajo')] = 'A1303'
datos.iloc[455, datos.columns.get_loc('trabajo')] = 'A1407'
datos.iloc[334, datos.columns.get_loc('trabajo')] = 'A0504'

In [241]:
datos.iloc[659, datos.columns.get_loc('categoria')] = 'A99'
datos.iloc[624, datos.columns.get_loc('categoria')] = 'A99'
datos.iloc[566, datos.columns.get_loc('categoria')] = 'A99'
datos.iloc[327, datos.columns.get_loc('categoria')] = 'A01'
datos.iloc[170, datos.columns.get_loc('categoria')] = 'A13'
datos.iloc[455, datos.columns.get_loc('categoria')] = 'A14'
datos.iloc[334, datos.columns.get_loc('categoria')] = 'A05'


In [240]:
datos[['trabajo', 'categoria']]

,trabajo,categoria
0,A9921,A99
1,A9921,A99
2,A9921,A99
3,A9921,A99
4,A1303,A13
...,...,...
717,A9921,A99
718,A9921,A99
719,A9921,A99
720,A9921,A99


In [685]:
#import json

#trabajos = { ... }

#with open("trabajos.json", "w") as archivo:
#    json.dump(mapeo_codigos_ocupaciones, archivo)

Exportación final

In [242]:
datos.to_csv('zoom1_limpio_final.csv', index=False)

In [689]:
mostrar_unicos('Carrera')

0             A1
1            A10
2             A2
3             A3
4             A4
5             A5
6             A6
7             A7
8             A8
9             A9
10    psicología
Name: Carrera, dtype: object

In [690]:
with open("mapeo_carrera.json", "w", encoding="utf-8") as f:
    json.dump(mapeo_carrera, f, ensure_ascii=False)

In [691]:
h = []
for cat, codigo in mapeo_codigos_categoria.items():
    a = codigo + ' = ' + cat
    print(a)
    h.append(a)
h.sort()

A99 = desconocido_99
A13 = Ã¡rea educativa_13
A07 = comercio virtual_7
A05 = comerciante sector formal_5
A04 = comerciante sector informal_4
A14 = profesionista independiente_14
A08 = atenciÃ³n a clientes_8
A20 = actividades elementales y de apoyo_20
A09 = administrativo_9
A06 = empresario_6
A11 = empleado gubernamental_11
A02 = artista_2
A21 = oficios_21
A18 = fÃ¡brica_18
A16 = Ã¡rea de la salud_16
A01 = actividades deportivas_1
A10 = gubernamental administrativo_10
A03 = chofer_3


In [692]:
h

['A01 = actividades deportivas_1',
 'A02 = artista_2',
 'A03 = chofer_3',
 'A04 = comerciante sector informal_4',
 'A05 = comerciante sector formal_5',
 'A06 = empresario_6',
 'A07 = comercio virtual_7',
 'A08 = atenciÃ³n a clientes_8',
 'A09 = administrativo_9',
 'A10 = gubernamental administrativo_10',
 'A11 = empleado gubernamental_11',
 'A13 = Ã¡rea educativa_13',
 'A14 = profesionista independiente_14',
 'A16 = Ã¡rea de la salud_16',
 'A18 = fÃ¡brica_18',
 'A20 = actividades elementales y de apoyo_20',
 'A21 = oficios_21',
 'A99 = desconocido_99']

In [693]:
csv_ocup = []
for ocup, codigo in mapeo_codigos_ocupaciones.items():
    a = codigo + ' = ' + ocup
    csv_ocup.append(a)
csv_ocup.sort()

In [694]:
csv_ocup

['A0101 = entrenador',
 'A0102 = instructor',
 'A0201 = actor',
 'A0202 = ballet',
 'A0203 = modelo',
 'A0204 = mÃºsico',
 'A0301 = camiones',
 'A0302 = taxista',
 'A0401 = bazar',
 'A0402 = flores y productos en un puesto no establecido',
 'A0403 = ropa',
 'A0404 = vendedor por cuenta propia (negocio propio)',
 'A0405 = venta de cigarros',
 'A0501 = taquero',
 'A0502 = vendedor de comida',
 'A0601 = asesor',
 'A0602 = consultor',
 'A0603 = fundaciÃ³n',
 'A0604 = gerente',
 'A0701 = vendedor de productos por internet (mercado pago, amazon, instagram, etc.)',
 'A0801 = atenciÃ³n a clientes',
 'A0802 = cajero',
 'A0803 = call center',
 'A0804 = promotor de seguros',
 'A0805 = recepcionista',
 'A0806 = vendedor de tiendas departamentales',
 'A0901 = reclutador',
 'A0902 = recursos humanos',
 'A1001 = sistemas',
 'A1101 = administrador',
 'A1102 = director general',
 'A1301 = dar clases',
 'A1302 = docente',
 'A1303 = estudiante',
 'A1304 = idiomas',
 'A1305 = investigador-profesor en univ

In [695]:
exportar_categorias = pd.DataFrame({'related': ['G'],
                                    'alive': ['_'],
                                    'new_code': ['ocupacion_nivel1'],
                                    'code': ['cagegoria']})
exportar_categorias

,related,alive,new_code,code
0,G,_,ocupacion_nivel1,cagegoria


In [696]:
for i in range(1, len(h) + 1):
    exportar_categorias['_' + str(i)] = h[i - 1]

exportar_categorias

,related,alive,new_code,code,_1,_2,_3,_4,_5,_6,...,_9,_10,_11,_12,_13,_14,_15,_16,_17,_18
0,G,_,ocupacion_nivel1,cagegoria,A01 = actividades deportivas_1,A02 = artista_2,A03 = chofer_3,A04 = comerciante sector informal_4,A05 = comerciante sector formal_5,A06 = empresario_6,...,A09 = administrativo_9,A10 = gubernamental administrativo_10,A11 = empleado gubernamental_11,A13 = Ã¡rea educativa_13,A14 = profesionista independiente_14,A16 = Ã¡rea de la salud_16,A18 = fÃ¡brica_18,A20 = actividades elementales y de apoyo_20,A21 = oficios_21,A99 = desconocido_99
